# Optimal Beam Width for English to Marathi Neural Machine Translation with Beam Search
## DATASCI W266: Natural Language Processing with Deep Learning
### Rohit Bakshi

This Jupyter notebook contains the code to the entire project.

## Dataset
Link: http://www.manythings.org/anki/  
File: mar-eng.zip

## Data Loading and Preprocessing

In [739]:
# import all libraries
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Input, Dense,Embedding, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras import Input, Model
from tensorflow.keras.models import Model,load_model, model_from_json
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from nltk.translate.bleu_score import sentence_bleu

In [704]:
# read from file location into Pandas Dataframe
file_path = f'C:\\Users\\rohit\\Downloads\\mar-eng\\mar.txt'
lines= pd.read_table(file_path, names=['eng','mar','metadata']).drop(['metadata'], axis=1)

In [705]:
# show Dataframe
lines

,eng,mar
0,Go.,जा.
1,Run!,पळ!
2,Run!,धाव!
3,Run!,पळा!
4,Run!,धावा!
...,...,...
43066,Just saying you don't like fish because of the...,हड्डींमुळे मासे आवडत नाही असं म्हणणं हे काय मा...
43067,The Japanese Parliament today officially elect...,आज जपानी संसदेने अधिकृतरित्या र्‍यौतारौ हाशिमो...
43068,Tom tried to sell his old VCR instead of throw...,टॉमने त्याचा जुना व्ही.सी.आर फेकून टाकण्याऐवजी...
43069,You can't view Flash content on an iPad. Howev...,आयपॅडवर फ्लॅश आशय बघता येत नाही. पण तुम्ही त्य...


In [706]:
#ensure dataframe shape is as expected
lines.shape

(43071, 2)

In [707]:
# lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.mar=lines.mar.apply(lambda x: x.lower())

In [708]:
# remove all quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.mar=lines.mar.apply(lambda x: re.sub("'", '', x))

In [709]:
# remove all special characters
exclude = set(string.punctuation) # Set of all special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.mar=lines.mar.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [710]:
# remove all numbers in English and Marathi
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.mar = lines.mar.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

In [711]:
# remove all extra whitespace
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.mar=lines.mar.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.mar=lines.mar.apply(lambda x: re.sub(" +", " ", x))

In [712]:
# add START_ and _END tokens to target sequences
lines.mar = lines.mar.apply(lambda x : 'START_ '+ x + ' _END')

In [713]:
# check sample
lines.sample(10)

,eng,mar
40296,ive never been to my grandfathers house,START_ मी माझ्या आजोबांच्या घरी कधीच गेलो नाही...
21555,tom was driving the bus,START_ ती बस टॉम चालवत होता _END
26220,i dont have any money now,START_ माझ्याकडे आता अजिबात पैसे नाहीयेत _END
15786,you cant understand,START_ तू समजू शकत नाहीस _END
4679,im an organist,START_ मी ऑर्गनिस्ट आहे _END
29557,i didnt drink tea yesterday,START_ काल मी चहा प्यायले नाही _END
16975,my salary is very low,START_ माझा पगार खूपच कमी आहे _END
33040,what did tom tell you about me,START_ टॉमने तुला माझ्याबद्दल काय सांगितलं _END
1769,they saw tom,START_ त्यांनी टॉमला पाहिलं _END
33182,your english has gotten better,START_ तुझी इंग्रजी सुधारली आहे _END


In [714]:
lines

,eng,mar
0,go,START_ जा _END
1,run,START_ पळ _END
2,run,START_ धाव _END
3,run,START_ पळा _END
4,run,START_ धावा _END
...,...,...
43066,just saying you dont like fish because of the ...,START_ हड्डींमुळे मासे आवडत नाही असं म्हणणं हे...
43067,the japanese parliament today officially elect...,START_ आज जपानी संसदेने अधिकृतरित्या र्‍यौतारौ...
43068,tom tried to sell his old vcr instead of throw...,START_ टॉमने त्याचा जुना व्हीसीआर फेकून टाकण्य...
43069,you cant view flash content on an ipad however...,START_ आयपॅडवर फ्लॅश आशय बघता येत नाही पण तुम्...


In [715]:
# create vocabulary of English and Marathi
# English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
# Marathi 
all_marathi_words=set()
for mar in lines.mar:
    for word in mar.split():
        if word not in all_marathi_words:
            all_marathi_words.add(word)

In [1308]:
# max Length of source sequence (English)
length_list=[]
for l in lines.eng:
    length_list.append(len(l.split(' ')))
max_length_src = np.max(length_list)
max_length_src

34

In [1309]:
# max Length of target sequence (Marathi)
length_list=[]
for l in lines.mar:
    length_list.append(len(l.split(' ')))
max_length_tar = np.max(length_list)
max_length_tar

37

In [718]:
# get unique tokens in both languages
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_marathi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_marathi_words)
num_encoder_tokens, num_decoder_tokens

(5794, 14037)

In [719]:
# padding for Neural Network Model to work
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

14038

In [720]:
# dictionary to map index to word
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [721]:
# dictionary to map index to word
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [722]:
lines = shuffle(lines)
lines.head(10)

,eng,mar
21692,were big fans of yours,START_ आम्ही तुमच्या मोठ्या फॅन आहोत _END
34598,open the door and let in the dog,START_ दरवाजा उघड व कुत्र्याला आत येऊ दे _END
32147,did you drink coffee yesterday,START_ काल तू कॉफी प्यायलीस का _END
17692,were learning french,START_ आम्ही फ्रेंच शिकत आहोत _END
39910,i think that tom will tell you the truth,START_ मला वाटतं की टॉम तुम्हाला खरं काय ते सा...
37087,my girlfriend is tall and beautiful,START_ माझी गर्लफ्रेंड उंच आणि सुंदर आहे _END
31113,i met tom when i was thirteen,START_ मी तेरा वर्षांचा असताना टॉमला भेटलो _END
26647,my dad doesnt like soccer,START_ माझ्या बाबांना फुटबॉल आवडत नाही _END
8373,tell tom yourself,START_ टॉमला स्वतः सांगा _END
7839,i really like you,START_ मला तुम्ही खरच आवडतात _END


In [723]:
# train - test split of 90/10
X, y = lines.eng, lines.mar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((38763,), (4308,))

In [724]:
lines.mar

21692            START_ आम्ही तुमच्या मोठ्या फॅन आहोत _END
34598        START_ दरवाजा उघड व कुत्र्याला आत येऊ दे _END
32147                  START_ काल तू कॉफी प्यायलीस का _END
17692                   START_ आम्ही फ्रेंच शिकत आहोत _END
39910    START_ मला वाटतं की टॉम तुम्हाला खरं काय ते सा...
                               ...                        
27591    START_ तुला काहीही चांगलं असं म्हणता येत नाही ...
297                                    START_ कसा आहे _END
5927                      START_ ते फ्रान्समध्ये होते _END
24189                START_ तो भिंतीला लागून उभा झाला _END
34741        START_ या तलवारीचा एक विचित्र इतिहास आहे _END
Name: mar, Length: 43071, dtype: object

### Batch Generator
This is used to create batches for loading data into the model and running experiments with decoding translations.

In [725]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

## Encoder - Decoder Model Architecture for Training

In [726]:
latent_dim = 50

In [727]:
# encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# only keep states and discard outputs
encoder_states = [state_h, state_c]

In [728]:
# initialize decoder with final states from encoder
# decoder for training
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens+1, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# define the model that will process encoder and decoder inputs into decoder outputs
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [729]:
# model hyperparameters
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [730]:
train_samples = len(X_train)
val_samples = len(X_test)
# model hyperparameters
batch_size = 128
epochs = 50

In [127]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

C:\Users\rohit\AppData\Local\Temp/ipykernel_17724/2108290721.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


Epoch 1/50
302/302 [==============================] - 258s 837ms/step - loss: 1.0379 - accuracy: 0.1744 - val_loss: 0.8920 - val_accuracy: 0.1795
Epoch 2/50
302/302 [==============================] - 226s 749ms/step - loss: 0.8804 - accuracy: 0.1965 - val_loss: 0.8525 - val_accuracy: 0.2056
Epoch 3/50
302/302 [==============================] - 228s 754ms/step - loss: 0.8412 - accuracy: 0.2166 - val_loss: 0.8210 - val_accuracy: 0.2296
Epoch 4/50
302/302 [==============================] - 227s 752ms/step - loss: 0.8071 - accuracy: 0.2453 - val_loss: 0.7911 - val_accuracy: 0.2561
Epoch 5/50
302/302 [==============================] - 226s 750ms/step - loss: 0.7714 - accuracy: 0.2619 - val_loss: 0.7630 - val_accuracy: 0.2697
Epoch 6/50
302/302 [==============================] - 227s 753ms/step - loss: 0.7390 - accuracy: 0.2791 - val_loss: 0.7386 - val_accuracy: 0.2857
Epoch 7/50
302/302 [==============================] - 226s 750ms/step - loss: 0.7091 - accuracy: 0.2978 - val_loss: 0.7154 -

#### save weights of the model

In [128]:
model.save_weights('nmt_weights.h5')

#### load saved weights

In [731]:
model.load_weights('nmt_weights.h5')

## Inference, Translating Sentences and Scoring Sentences

### Encoder Model and Decoder Model for Inference

In [732]:
# encode input sequence to get final internal states
encoder_model = Model(encoder_inputs, encoder_states)

# initialize decoder - need to hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 

# the initial states to the states from the previous time step to predict the next word
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# dense softmax layer to generate prob distribution over the target vocabulary (Marathi)
decoder_outputs2 = decoder_dense(decoder_outputs2) 

# final decoder model for inference used below in decode_sequence functions
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

### Beam Search Decoder with sample sequences

In [1297]:
def beam_search_decode_sequence(input_seq, k):
    # encode the input as state vectors
    states_value = encoder_model.predict(input_seq)
    # generate empty target sequence of length 1
    target_seq = np.zeros((1,1))
    # populate the first character of target sequence with the start character
    target_seq[0, 0] = target_token_index['START_']

    stop_condition = False
    decoded_sentence = ''
    sequences = [[list(), 0.0]]
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        
        # take k highest probabilites
        ind = np.argpartition(output_tokens[0, -1, :], -k)[-k:]
        ind = ind[np.argsort(output_tokens[0, -1, :][ind])]
        row = output_tokens[0, -1, :][ind]
        all_candidates = list()
        # expand each current candidate
        for i in range(len(sequences)):
            seq, score = sequences[i]
            for j in range(len(row)):
                candidate = [seq + [reverse_target_char_index[ind[j]]], score - log(row[j])]
                all_candidates.append(candidate)
            # order all candidates by score
        ordered = sorted(all_candidates, key=lambda tup:tup[1])
        # select k best
        sequences = ordered[:k]
        
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # stop condition: either hit max length or find stop character
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # update the target sequence (of length 1)
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # update states
        states_value = [h, c]

    decoded_sentences_and_scores = []
    decoded_sentences = []
    for seq, score in sequences:
        decoded_sentences_and_scores.append((' '.join(seq[:-1]), score))
        decoded_sentences.append(' '.join(seq[:-1]))
    
    #print("Top Sentences")
    #print(decoded_sentences)
    #print("\n")
    res = decoded_sentences[0]
    return decoded_sentences

### Baseline Greedy Decoder with sample sequences

In [632]:
def decode_sequence(input_seq):
    # encode the input as state vectors
    states_value = encoder_model.predict(input_seq)
    # generate empty target sequence of length 1
    target_seq = np.zeros((1,1))
    # populate the first character of target sequence with the start character
    target_seq[0, 0] = target_token_index['START_']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # stop condition: either hit max length or find stop character
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # update the target sequence (of length 1)
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # update states
        states_value = [h, c]

    return decoded_sentence

In [784]:
def sentence_to_lists(sentence_list):
    res = []
    for sentence in sentence_list:
        res.append(sentence.split())
    return res
        
sentence_to_lists(['मी एक सँडविच विकत घेतलं', 'मी एक चादर विकत घेतलं', 'मी एक सँडविच विकत घेतला'])

[['मी', 'एक', 'सँडविच', 'विकत', 'घेतलं'],
 ['मी', 'एक', 'चादर', 'विकत', 'घेतलं'],
 ['मी', 'एक', 'सँडविच', 'विकत', 'घेतला']]

### BLEU Score

In [854]:
# calculate best BLEU score for all sentences in the beam width
def calculate_bleu_for_beam(reference, sentences):
    sentences = sentence_to_lists(sentences)
    reference = [reference.strip()]
    reference = sentence_to_lists(reference)
    #print(reference)
    bleu = []
    for sentence in sentences:
        #print(sentence)
        #unigrams
        BLEUscore = sentence_bleu(reference, sentence, weights = (1,0))
        #print(BLEUscore)
        bleu.append(BLEUscore)
    return max(bleu)

calculate_bleu_for_beam("मी एक सँडविच विकत घेतलं", ['मी एक सँडविच विकत घेतलं', 'मी एक चादर विकत घेतलं', 'मी एक सँडविच विकत घेतला'])

1.0

In [931]:
# return the best sentence with the highest BLEU score
def best_beam(reference, sentences):
    sentences = sentence_to_lists(sentences)
    reference = [reference.strip()]
    reference = sentence_to_lists(reference)
    #print(reference)
    bleu = 0
    res = None
    for sentence in sentences:
        #print(sentence)
        #unigrams
        BLEUscore = sentence_bleu(reference, sentence, weights = (1,0))
        #print(BLEUscore)
        if BLEUscore > bleu:
            bleu = BLEUscore
            res = sentence
    return " ".join(res)

best_beam("मी एक सँडविच विकत घेतलं", ['मी एक सँडविच विकत घेतलं', 'मी एक चादर विकत घेतलं', 'मी एक सँडविच विकत घेतला'])

'मी एक सँडविच विकत घेतलं'

In [882]:
# calculate the BLEU score for the greedy baseline model
def calculate_bleu(reference, sentence):
    reference = [reference.strip()]
    reference = sentence_to_lists(reference)
    sentence = sentence_to_lists([sentence])
    sentence = sentence[0]
    BLEUscore = sentence_bleu(reference, sentence, weights = (1,0))
    return BLEUscore
calculate_bleu("मी एक सँडविच विकत घेतलं","मी एक सँडविच विकत घेतलं ")

1.0

## Training Dataset Evaluation

In [1284]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [1285]:
n = 3

### Training Experiments
Run the greedy and beam search decoder for 100 examples in the training dataset with varying beam width (k) and show the difference in total BLEU score between the beam search model and greedy model as well as the total time of execution.

In [1322]:
import time
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1
n = 200
total_diff = 0
tic = time.perf_counter()
for x in range (0,100):
    k+=1
    (input_seq, actual_output), _ = next(train_gen)
    input_sentence = X_train[k:k+1].values[0]
    decoded_sentence = decode_sequence(input_seq)[:-4]
    beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
    actual_sentence = y_train[k:k+1].values[0][6:-4]
    print('Input English sentence:', input_sentence)
    #print('Actual Marathi Translation:', actual_sentence)
    #print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
    #print('Predicted Marathi Translation:', decoded_sentence)

    beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
    score = calculate_bleu(actual_sentence, decoded_sentence)
    diff = beam_score - score
    total_diff += diff
    #print(beam_score)
    #print(score)
toc = time.perf_counter()
print(n, total_diff)
print(f"Time for execution: {toc - tic:0.4f} seconds")

Input English sentence: i bought a sandwich
Input English sentence: do you think its impossible to finish this by five oclock
Input English sentence: raise your hand before you answer
Input English sentence: he fell asleep while reading a book
Input English sentence: how did you find it
Input English sentence: the bus fares have been raised by percent
Input English sentence: do you ever think about that girl
Input English sentence: tom works in a factory
Input English sentence: weve learned a lot too
Input English sentence: lets stay here tonight
Input English sentence: she shut the door and went upstairs
Input English sentence: walt whitman is my favorite american poet
Input English sentence: what does tom see in mary
Input English sentence: is this your cap
Input English sentence: i need to do this
Input English sentence: camels are the ships of the deserts
Input English sentence: look someplace else
Input English sentence: did you know that tom has three children
Input English sente

In [1321]:
import time
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1
n = 100
total_diff = 0
tic = time.perf_counter()
for x in range (0,100):
    k+=1
    (input_seq, actual_output), _ = next(train_gen)
    input_sentence = X_train[k:k+1].values[0]
    decoded_sentence = decode_sequence(input_seq)[:-4]
    #beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
    actual_sentece = y_train[k:k+1].values[0][6:-4]
    print('Input English sentence:', input_sentence)
    #print('Actual Marathi Translation:', actual_sentence)
    #print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
    #print('Predicted Marathi Translation:', decoded_sentence)

    #beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
    #score = calculate_bleu(actual_sentence, decoded_sentence)
    #diff = beam_score - score
    #total_diff += diff
    #print(beam_score)
    #print(score)
toc = time.perf_counter()
print(n, total_diff)
print(f"Time for execution: {toc - tic:0.4f} seconds")

Input English sentence: i bought a sandwich
Input English sentence: do you think its impossible to finish this by five oclock
Input English sentence: raise your hand before you answer
Input English sentence: he fell asleep while reading a book
Input English sentence: how did you find it
Input English sentence: the bus fares have been raised by percent
Input English sentence: do you ever think about that girl
Input English sentence: tom works in a factory
Input English sentence: weve learned a lot too
Input English sentence: lets stay here tonight
Input English sentence: she shut the door and went upstairs
Input English sentence: walt whitman is my favorite american poet
Input English sentence: what does tom see in mary
Input English sentence: is this your cap
Input English sentence: i need to do this
Input English sentence: camels are the ships of the deserts
Input English sentence: look someplace else
Input English sentence: did you know that tom has three children
Input English sente

### Training Examples - Individual Assessment of sentences

In [1143]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मी एक सँडविच विकत घेतलं', 'मी एक चादर विकत घेतलं', 'मी एक सँडविच विकत घेतला', 'मी एक सँडविच पाठवलं घेतलं', 'मी एक सँडविच आणलं घेतलं', 'मी एक सँडविच खाल्लं घेतलं', 'मी एक चादर विकत घेतला', 'मी एक सँडविच बघितलं घेतलं', 'मी एक सँडविच विकत घेईन', 'मी एक चादर पाठवलं घेतलं']


Input English sentence: i bought a sandwich
Actual Marathi Translation:  मी एक सँडविच विकत घेतलं 
Beam Predicted Marathi Translation: मी एक सँडविच विकत घेतलं
Predicted Marathi Translation:  मी एक सँडविच विकत घेतलं 
1.0
1.0


In [1144]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तर तर तर तू खरच किती मोठं विचार का आहे', 'तर तर तर तू किती किती मोठं विचार का आहे', 'तर तर लोकसंख्या तू खरच किती मोठं विचार का आहे', 'तर तर तर लोकसंख्या खरच किती मोठं विचार का आहे', 'तर तर लोकसंख्या तू किती किती मोठं विचार का आहे', 'तर तर तर लोकसंख्या किती किती मोठं विचार का आहे', 'तर तर तू तू खरच किती मोठं विचार का आहे', 'तर तर तू तू किती किती मोठं विचार का आहे', 'तर तर तर तुला खरच किती मोठं विचार का आहे', 'तर तर तर तुला किती किती मोठं विचार का आहे']


Input English sentence: do you think its impossible to finish this by five oclock
Actual Marathi Translation:  हे पाच वाजेपर्यंत संपवणं तुला अशक्य वाटतंय का 
Beam Predicted Marathi Translation: तर तर तर तुला खरच किती मोठं विचार का आहे
Predicted Marathi Translation:  तर तर तर तू खरच किती मोठं विचार का आहे 
0.2
0.10000000000000002


In [1145]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['उत्तर द्यायच्या आधी हात वर करा', 'उत्तर द्यायच्या आधी हात वर कर', 'उत्तर देण्याअगोदर आधी हात वर करा', 'उत्तर देण्याअगोदर आधी हात वर कर', 'उत्तर द्यायच्या फोन हात वर करा', 'उत्तर द्यायच्या हात हात वर करा', 'माझ्या द्यायच्या आधी हात वर करा', 'उत्तर द्यायच्या आधी फोन वर करा', 'तुझं द्यायच्या आधी हात वर करा', 'उत्तर द्यायच्या फोन हात वर कर']


Input English sentence: raise your hand before you answer
Actual Marathi Translation:  उत्तर देण्याअगोदर हात वर कर 
Beam Predicted Marathi Translation: उत्तर देण्याअगोदर आधी हात वर कर
Predicted Marathi Translation:  उत्तर द्यायच्या आधी हात वर करा 
0.8333333333333334
0.5


In [1146]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तो पुस्तक वाचत होता', 'तो पुस्तक वाचत असताना', 'त्याने पुस्तक वाचत होता', 'ते पुस्तक वाचत होता', 'तो पत्र वाचत होता', 'त्याने पुस्तक वाचत असताना', 'तो पुस्तक वाचत होता', 'ते पुस्तक वाचत असताना', 'तो वाचत वाचत होता', 'तो पुस्तक वाचतावाचता होता']


Input English sentence: he fell asleep while reading a book
Actual Marathi Translation:  तो एक पुस्तक वाचतावाचता झोपून गेला 
Beam Predicted Marathi Translation: तो पुस्तक वाचतावाचता होता
Predicted Marathi Translation:  तो पुस्तक वाचत होता 
0.45489799478447507
0.3032653298563167


In [1147]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तुम्हाला कसं सापडलं', 'तुला कसं सापडलं', 'तुम्हाला काय सापडलं', 'तुला काय सापडलं', 'तुम्हाला कसं शिकवलं', 'तुला कसं शिकवलं', 'तुम्हाला सापडलं सापडलं', 'तुला सापडलं सापडलं', 'तुम्हाला कसं सापडला', 'तुला कसं सापडला']


Input English sentence: how did you find it
Actual Marathi Translation:  तुम्हाला कशी सापडली 
Beam Predicted Marathi Translation: तुम्हाला कसं सापडलं
Predicted Marathi Translation:  तुम्हाला कसं सापडलं 
0.3333333333333333
0.3333333333333333


In [1148]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['बसचे भाडे टक्क्यांनी वाढवले आहे', 'व्यसनी भाडे टक्क्यांनी वाढवले आहे', 'टेप भाडे टक्क्यांनी वाढवले आहे', 'गावाच्या भाडे टक्क्यांनी वाढवले आहे', 'कॅलिफोर्नियामध्ये भाडे टक्क्यांनी वाढवले आहे', 'आरोग्यापेक्षा भाडे टक्क्यांनी वाढवले आहे', 'यामामोतोकडे भाडे टक्क्यांनी वाढवले आहे', 'एक्स्टर्नल भाडे टक्क्यांनी वाढवले आहे', 'द्वाराजवळ भाडे टक्क्यांनी वाढवले आहे', 'हॅलोवीन भाडे टक्क्यांनी वाढवले आहे']


Input English sentence: the bus fares have been raised by percent
Actual Marathi Translation:  बसचे भाडे टक्क्यांनी वाढवले आहे 
Beam Predicted Marathi Translation: बसचे भाडे टक्क्यांनी वाढवले आहे
Predicted Marathi Translation:  बसचे भाडे टक्क्यांनी वाढवले आहे 
1.0
1.0


In [1149]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तुम्ही त्याला कधी मुलीबद्दल विचार करतो का', 'तू त्याला कधी मुलीबद्दल विचार करतो का', 'तुम्ही त्याला कधी मुलीबद्दल विचार करत का', 'तू त्याला कधी मुलीबद्दल विचार करत का', 'तुम्ही त्याला कधी मुलीबद्दल विचार करता का', 'तू त्याला कधी मुलीबद्दल विचार करता का', 'तुम्ही त्याला कधी अमेरिकन विचार करतो का', 'तू त्याला कधी अमेरिकन विचार करतो का', 'तुम्ही त्यांना कधी मुलीबद्दल विचार करतो का', 'तू त्यांना कधी मुलीबद्दल विचार करतो का']


Input English sentence: do you ever think about that girl
Actual Marathi Translation:  तू कधी त्या मुलीबद्दल विचार करतेस का 
Beam Predicted Marathi Translation: तू त्याला कधी मुलीबद्दल विचार करतो का
Predicted Marathi Translation:  तुम्ही त्याला कधी मुलीबद्दल विचार करतो का 
0.7142857142857143
0.5714285714285714


In [1150]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['टॉम एका कारखान्यात काम करतो', 'टॉम एका फॅक्टरीत काम करतो', 'टॉम एका कारखान्यात काम करते', 'टॉम एका फॅक्टरीत काम करते', 'टॉम एका एका काम करतो', 'टॉम एका शहरी काम करतो', 'टॉम एका बेकरीत काम करतो', 'टॉम एका छोट्या काम करतो', 'टॉम एका बँकेत काम करतो', 'टॉम एका एका काम करते']


Input English sentence: tom works in a factory
Actual Marathi Translation:  टॉम एका कारखान्यात काम करतो 
Beam Predicted Marathi Translation: टॉम एका कारखान्यात काम करतो
Predicted Marathi Translation:  टॉम एका कारखान्यात काम करतो 
1.0
1.0


In [1151]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['आम्ही भरपूर काही आहेत आहोत हे', 'आम्ही भरपूर काही शिकलो आहोत हे', 'आम्ही भरपूर कॉफी आहेत आहोत हे', 'आम्ही भरपूर कॉफी शिकलो आहोत हे', 'आपण भरपूर काही आहेत आहोत हे', 'आपण भरपूर काही शिकलो आहोत हे', 'आम्ही भरपूर तीस आहेत आहोत हे', 'आम्ही भरपूर तीस शिकलो आहोत हे', 'आम्ही भरपूर काही आहेत आहोत हे', 'आपण भरपूर कॉफी आहेत आहोत हे']


Input English sentence: weve learned a lot too
Actual Marathi Translation:  आपणही भरपूर काही शिकलो आहोत 
Beam Predicted Marathi Translation: आम्ही भरपूर काही शिकलो आहोत हे
Predicted Marathi Translation:  आम्ही भरपूर काही आहेत आहोत हे 
0.6666666666666666
0.5


In [1152]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['आज रात्री इथे राहूया', 'आज रात्री इथे जाऊया', 'इथे रात्री इथे राहूया', 'इथे रात्री इथे जाऊया', 'आज रात्री इथून राहूया', 'आज रात्री इथे नसतो', 'आज रात्री इथून जाऊया', 'आज रात्री इथे राहिला', 'आज रात्री इथे गेलं', 'आज रात्री इथे राहू']


Input English sentence: lets stay here tonight
Actual Marathi Translation:  आज रात्री इथे राहूया 
Beam Predicted Marathi Translation: आज रात्री इथे राहूया
Predicted Marathi Translation:  आज रात्री इथे राहूया 
1.0
1.0


In [1153]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['त्यांनी दरवाजा बंद केला का', 'त्यांनी दरवाजा बंद केलं का', 'त्यांनी जे बंद केला का', 'त्यांनी जे बंद केलं का', 'त्यांनी खरं बंद केला का', 'तिने दरवाजा बंद केला का', 'टॉमने दरवाजा बंद केला का', 'त्यांनी दरवाजा बंद करून का', 'त्यांनी खरं बंद केलं का', 'तिने दरवाजा बंद केलं का']


Input English sentence: she shut the door and went upstairs
Actual Marathi Translation:  ती दार बंद करून वर गेली 
Beam Predicted Marathi Translation: त्यांनी दरवाजा बंद करून का
Predicted Marathi Translation:  त्यांनी दरवाजा बंद केला का 
0.3274923012311928
0.1637461506155964


In [1154]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['माझा सर्वात आवडता कवी नाव लेखक आहे', 'माझा सर्वात आवडता कवी सर्वात लेखक आहे', 'माझा आवडता आवडता कवी नाव लेखक आहे', 'माझा आवडता आवडता कवी सर्वात लेखक आहे', 'माझा सर्वात सर्वात कवी नाव लेखक आहे', 'माझा सर्वात सर्वात कवी सर्वात लेखक आहे', 'माझा आवडता सर्वात कवी नाव लेखक आहे', 'माझा आवडता सर्वात कवी सर्वात लेखक आहे', 'माझा सर्वात आवडता कवी नाव झोपलेली आहे', 'माझा सर्वात आवडता कवी सर्वात झोपलेली आहे']


Input English sentence: walt whitman is my favorite american poet
Actual Marathi Translation:  माझा सर्वात आवडता अमेरिकन कवी वॉल्ट व्हिटमन आहे 
Beam Predicted Marathi Translation: माझा सर्वात आवडता कवी नाव लेखक आहे
Predicted Marathi Translation:  माझा सर्वात आवडता कवी नाव लेखक आहे 
0.6191984998215584
0.6191984998215584


In [1155]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['टॉमला मेरीत काय दिसतं', 'टॉमला मेरीमध्ये काय दिसतं', 'टॉमला मेरीत काय होता', 'टॉम मेरीत काय दिसतं', 'टॉमला मेरीमध्ये काय होता', 'टॉमला काय काय दिसतं', 'टॉम मेरीमध्ये काय दिसतं', 'टॉम मेरीत काय होता', 'टॉमला मेरीत काय दिसत', 'टॉमला बॉस्टनमध्ये काय दिसतं']


Input English sentence: what does tom see in mary
Actual Marathi Translation:  टॉमला मेरीत काय दिसतं 
Beam Predicted Marathi Translation: टॉमला मेरीत काय दिसतं
Predicted Marathi Translation:  टॉमला मेरीत काय दिसतं 
1.0
1.0


In [1156]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['हे तुझी टोपी आहे का', 'हे तुमची टोपी आहे का', 'हे तुझं टोपी आहे का', 'ही तुझी टोपी आहे का', 'हे तुमचं टोपी आहे का', 'ही तुमची टोपी आहे का', 'ही तुझं टोपी आहे का', 'ही तुमचं टोपी आहे का', 'हे घर टोपी आहे का', 'ही घर टोपी आहे का']


Input English sentence: is this your cap
Actual Marathi Translation:  ही तुझी टोपी आहे का 
Beam Predicted Marathi Translation: ही तुझी टोपी आहे का
Predicted Marathi Translation:  हे तुझी टोपी आहे का 
1.0
0.8


In [1157]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मला हे करायलाच पाहिजे', 'मला या करायलाच पाहिजे', 'मला हे गरज पाहिजे', 'मला हे करायचं पाहिजे', 'मला हे लागेल पाहिजे', 'मला याबद्दल करायलाच पाहिजे', 'मला या गरज पाहिजे', 'मला या करायचं पाहिजे', 'मला या लागेल पाहिजे', 'मला हे करायची पाहिजे']


Input English sentence: i need to do this
Actual Marathi Translation:  मला हे करायलाच पाहिजे 
Beam Predicted Marathi Translation: मला हे करायलाच पाहिजे
Predicted Marathi Translation:  मला हे करायलाच पाहिजे 
1.0
1.0


In [1158]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['उंट म्हणजे वाळवंटाची जहाजंच', 'गरुड म्हणजे वाळवंटाची जहाजंच', 'म्हणजे म्हणजे वाळवंटाची जहाजंच', 'लंडन म्हणजे वाळवंटाची जहाजंच', 'डोंगर म्हणजे वाळवंटाची जहाजंच', 'वरून म्हणजे वाळवंटाची जहाजंच', 'समुद्र म्हणजे वाळवंटाची जहाजंच', 'आकाशात म्हणजे वाळवंटाची जहाजंच', 'आकाश म्हणजे वाळवंटाची जहाजंच', 'उंट म्हणजे म्हणजे जहाजंच']


Input English sentence: camels are the ships of the deserts
Actual Marathi Translation:  उंट म्हणजे वाळवंटाची जहाजंच 
Beam Predicted Marathi Translation: उंट म्हणजे वाळवंटाची जहाजंच
Predicted Marathi Translation:  उंट म्हणजे वाळवंटाची जहाजंच 
1.0
1.0


In [1159]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['दुसरीकडे कुठेतरी शोध', 'कुठेतरी कुठेतरी शोध', 'देव कुठेतरी शोध', 'सध्या कुठेतरी शोध', 'दुसरीकडे कुठेतरी राहा', 'पाऊस कुठेतरी शोध', 'जोरदार कुठेतरी शोध', 'प्रकाश कुठेतरी शोध', 'उद्या कुठेतरी शोध', 'कुठेतरी कुठेतरी राहा']


Input English sentence: look someplace else
Actual Marathi Translation:  दुसरीकडे कुठेतरी शोध 
Beam Predicted Marathi Translation: दुसरीकडे कुठेतरी शोध
Predicted Marathi Translation:  दुसरीकडे कुठेतरी शोध 
1.0
1.0


In [1160]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['टॉमकडे तीन मुलं आहेत का', 'टॉमकडे तीन मुलं आहेत का', 'टॉमकडे तीन मुलं विकत का', 'टॉमकडे तीन मुलं आहेत का', 'टॉमकडे तीन मुलं आहेत का', 'टॉमला तीन मुलं आहेत का', 'टॉमकडे तीन मुलं विकत का', 'टॉमकडे तीन मुलं आहेत का', 'टॉमला तीन मुलं आहेत का', 'टॉमकडे तीन मुलं आहेत हे']


Input English sentence: did you know that tom has three children
Actual Marathi Translation:  टॉमला तीन मुलं आहेत हे तुला माहीत होतं का 
Beam Predicted Marathi Translation: टॉमला तीन मुलं आहेत का
Predicted Marathi Translation:  टॉमकडे तीन मुलं आहेत का 
0.44932896411722156
0.35946317129377725


In [1161]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मला माझी पिशवी द्या', 'मला माझी बॅग द्या', 'मला माझी पिशवी दे', 'मला माझी बॅग दे', 'मला माझ्या पिशवी द्या', 'मला माझ्या बॅग द्या', 'मला माझ्या पिशवी दे', 'मला माझ्या बॅग दे', 'मला माझे पिशवी द्या', 'माझी माझी पिशवी द्या']


Input English sentence: give me my bag
Actual Marathi Translation:  मला माझी पिशवी दे 
Beam Predicted Marathi Translation: मला माझी पिशवी दे
Predicted Marathi Translation:  मला माझी पिशवी द्या 
1.0
0.75


In [1162]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['ते तुमचं कप नाहीये', 'ते तुझं कप नाहीये', 'ते तुमचं उत्तर नाहीये', 'ते तुझं उत्तर नाहीये', 'ते तुमचं कप नाही', 'तो तुमचं कप नाहीये', 'ते तुझं कप नाही', 'ती तुमचं कप नाहीये', 'तो तुझं कप नाहीये', 'ती तुझं कप नाहीये']


Input English sentence: thats not your cup
Actual Marathi Translation:  ते तुमचं कप नाही आहे 
Beam Predicted Marathi Translation: ते तुमचं कप नाही
Predicted Marathi Translation:  ते तुमचं कप नाहीये 
0.7788007830714049
0.5841005873035536


In [1163]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['टॉमने त्याचा हेडसेट काढला', 'टॉमने स्वतःचा हेडसेट काढला', 'टॉमने स्वतःचं हेडसेट काढला', 'टॉमने त्याचं हेडसेट काढला', 'टॉमने त्याचा चेहरा काढला', 'टॉमने त्याचा स्वतःचा काढला', 'टॉमने आपला हेडसेट काढला', 'टॉमने स्वतःच्या हेडसेट काढला', 'टॉमने त्याचा त्याचा काढला', 'टॉमने त्याचा पासवर्ड काढला']


Input English sentence: tom took off his headset
Actual Marathi Translation:  टॉमने त्याचा हेडसेट काढला 
Beam Predicted Marathi Translation: टॉमने त्याचा हेडसेट काढला
Predicted Marathi Translation:  टॉमने त्याचा हेडसेट काढला 
1.0
1.0


In [1164]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['टॉमला तीन वर्षांची आले आहे', 'टॉमला तीन वर्षांची फ्रेंच आहे', 'टॉमला तीन वर्षांपूर्वी आले आहे', 'टॉमला तीन वर्षांपूर्वी फ्रेंच आहे', 'टॉमला तीन वर्ष आले आहे', 'टॉमला तीन वर्ष फ्रेंच आहे', 'टॉमला तीन वर्षांची आलो आहे', 'टॉमला तीन वर्षांपूर्वी आलो आहे', 'टॉमला तीन वर्षांची करतो आहे', 'टॉमला तीन वर्षांची झाली आहे']


Input English sentence: tom was sentenced to three years
Actual Marathi Translation:  टॉमला तीन वर्षांची शिक्षा सुनावण्यात आली 
Beam Predicted Marathi Translation: टॉमला तीन वर्षांची आले आहे
Predicted Marathi Translation:  टॉमला तीन वर्षांची आले आहे 
0.49123845184678916
0.49123845184678916


In [1165]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तू जुगार का खेळतोस', 'जुगार जुगार का खेळतोस', 'तू जुगार का खळतेस', 'तू जुगार का खेळता', 'तुम्ही जुगार का खेळतोस', 'जुगार जुगार का खळतेस', 'जुगार जुगार का खेळता', 'तू जुगार का खेळतेस', 'तू जुगार का खळतोस', 'तुम्ही जुगार का खळतेस']


Input English sentence: why do you gamble
Actual Marathi Translation:  तू जुगार का खळतेस 
Beam Predicted Marathi Translation: तू जुगार का खळतेस
Predicted Marathi Translation:  तू जुगार का खेळतोस 
1.0
0.75


In [1166]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मी साखर विकत घ्यायला विसरलो', 'मी साखर विकत घ्यायला विसरले', 'मी आणायला विकत घ्यायला विसरलो', 'मी साखर विकत घेतला विसरलो', 'मी ऑस्ट्रेलियाला विकत घ्यायला विसरलो', 'मी साखर आणायला घ्यायला विसरलो', 'मी बघायची विकत घ्यायला विसरलो', 'मी साखर शिकलो घ्यायला विसरलो', 'मी तुला विकत घ्यायला विसरलो', 'मी साखर विकत घेतली विसरलो']


Input English sentence: i forgot to buy sugar
Actual Marathi Translation:  मी साखर विकत घ्यायला विसरलो 
Beam Predicted Marathi Translation: मी साखर विकत घ्यायला विसरलो
Predicted Marathi Translation:  मी साखर विकत घ्यायला विसरलो 
1.0
1.0


In [1167]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तुझ्या आईची डोकं आहे का', 'तुझं आईची डोकं आहे का', 'तुमचं आईची डोकं आहे का', 'तुझ्या अजूनही डोकं आहे का', 'तुझ्या आईची मत आहे का', 'तुझं अजूनही डोकं आहे का', 'तुझ्या नाकातून डोकं आहे का', 'तुमचं अजूनही डोकं आहे का', 'तुमच्या आईची डोकं आहे का', 'तुझं आईची मत आहे का']


Input English sentence: does your head still hurt
Actual Marathi Translation:  तुमचं डोकं अजूनही दुखतंय का 
Beam Predicted Marathi Translation: तुमचं अजूनही डोकं आहे का
Predicted Marathi Translation:  तुझ्या आईची डोकं आहे का 
0.8
0.4


In [1168]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['आपण एकत्र प्रवास करत आहोत', 'आम्ही एकत्र प्रवास करत आहोत', 'आपण एकत्र प्रवास केला आहोत', 'आम्ही एकत्र प्रवास केला आहोत', 'एकत्र एकत्र प्रवास करत आहोत', 'आपण एकत्र प्रवास करत होतो', 'आम्ही एकत्र प्रवास करत होतो', 'आपण एकत्र प्रवास करत आहात', 'आम्ही एकत्र प्रवास करत आहात', 'एकत्र एकत्र प्रवास केला आहोत']


Input English sentence: were traveling together
Actual Marathi Translation:  आम्ही एकत्र प्रवास करत आहोत 
Beam Predicted Marathi Translation: आम्ही एकत्र प्रवास करत आहोत
Predicted Marathi Translation:  आपण एकत्र प्रवास करत आहोत 
1.0
0.8


In [1169]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['स्त्रिया पुरुषांपेक्षा कमी कमी आहेत', 'स्त्रिया पुरुषांपेक्षा कमी कमवतात आहेत', 'स्त्रिया पुरुषांपेक्षा कमी आहेत आहेत', 'स्त्रिया पुरुषांपेक्षा कमी असतात आहेत', 'स्त्रिया स्त्रिया कमी कमी आहेत', 'स्त्रिया पेक्षा कमी कमी आहेत', 'स्त्रिया पुरुषांपेक्षा कमी कमी आहे', 'स्त्रिया पुरुषांपेक्षा कमी खातात आहेत', 'स्त्रिया पुरुषांपेक्षा कमी लोकं आहेत', 'सैनिक पुरुषांपेक्षा कमी कमी आहेत']


Input English sentence: women earn less than men
Actual Marathi Translation:  स्त्रिया पुरुषांपेक्षा कमी कमवतात 
Beam Predicted Marathi Translation: स्त्रिया पुरुषांपेक्षा कमी कमवतात आहेत
Predicted Marathi Translation:  स्त्रिया पुरुषांपेक्षा कमी कमी आहेत 
0.8
0.6


In [1170]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['हे तुमचं घर आहे का', 'हे तुझं घर आहे का', 'हे घर घर आहे का', 'तुझं तुमचं घर आहे का', 'तुझं तुझं घर आहे का', 'तुझी तुमचं घर आहे का', 'तुमचं तुमचं घर आहे का', 'तुझं घर घर आहे का', 'तुमची तुमचं घर आहे का', 'तुझी तुझं घर आहे का']


Input English sentence: is this your house
Actual Marathi Translation:  हे तुझं घर आहे का 
Beam Predicted Marathi Translation: हे तुझं घर आहे का
Predicted Marathi Translation:  हे तुमचं घर आहे का 
1.0
0.8


In [1171]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मी दिवसाचे € कमवते', 'मी दिवसाचे € कमावतो', 'मी दिवसाचे € कमावते', 'मी दिवसाचे € रहायचा', 'मी दिवसाचे € काढायला', 'मी एक € कमवते', 'मी एक € कमावतो', 'मी एक € कमावते', 'मी दिवसाचे € समजला', 'मी दिवसाचे € लिहायचा']


Input English sentence: i make € a day
Actual Marathi Translation:  मी दिवसाचे € कमावतो 
Beam Predicted Marathi Translation: मी दिवसाचे € कमावतो
Predicted Marathi Translation:  मी दिवसाचे € कमवते 
1.0
0.75


In [1172]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मी महामार्ग पकडला', 'मी ब्राजिलहून पकडला', 'मी जोखीम पकडला', 'मी ब्राजिलपासून पकडला', 'मी महामार्ग मागवले', 'मी पापींबरोबर पकडला', 'मी ट्विटर पकडला', 'मी महामार्ग _END', 'मी कॅव्हिआर पकडला', 'मी महामार्ग बनवतो']


Input English sentence: i took highway
Actual Marathi Translation:  मी महामार्ग पकडला 
Beam Predicted Marathi Translation: मी महामार्ग पकडला
Predicted Marathi Translation:  मी महामार्ग पकडला 
1.0
1.0


In [1173]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मी माझं घड्याळ पाहिलं नाहीये', 'मी माझं घर पाहिलं नाहीये', 'मी अजूनपर्यंत घड्याळ पाहिलं नाहीये', 'मी माझं घड्याळ पाहिलं नाही', 'मी टॉमला घड्याळ पाहिलं नाहीये', 'मी चुकीच्या घड्याळ पाहिलं नाहीये', 'मी माझं मत पाहिलं नाहीये', 'मी माझं कधीही पाहिलं नाहीये', 'मी माझं कधीच पाहिलं नाहीये', 'मी अजूनपर्यंत घर पाहिलं नाहीये']


Input English sentence: i wont throw rocks at anybody
Actual Marathi Translation:  मी कोणावरही दगड टाकणार नाही 
Beam Predicted Marathi Translation: मी माझं घड्याळ पाहिलं नाही
Predicted Marathi Translation:  मी माझं घड्याळ पाहिलं नाहीये 
0.4
0.2


In [1174]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['हे पुस्तक नवीन आहे', 'हे पुस्तक एक आहे', 'हे पुस्तक काळी आहे', 'हे पुस्तक पुस्तक आहे', 'ते पुस्तक नवीन आहे', 'हे पुस्तक एकदम आहे', 'ही पुस्तक नवीन आहे', 'हा पुस्तक नवीन आहे', 'हे पुस्तक त्या आहे', 'हे पुस्तक बॉस्टन आहे']


Input English sentence: this book is very new
Actual Marathi Translation:  हे पुस्तक एकदम नवीन आहे 
Beam Predicted Marathi Translation: हे पुस्तक नवीन आहे
Predicted Marathi Translation:  हे पुस्तक नवीन आहे 
0.7788007830714049
0.7788007830714049


In [1175]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['त्याला मला विकशील का', 'त्याला मला केलंस का', 'त्याला मला करता का', 'तिला मला विकशील का', 'त्याला मला विकाल का', 'तिला मला केलंस का', 'त्याला मला सांगाल का', 'त्यांना मला विकशील का', 'त्याला मला म्हणतात का', 'त्यांना मला केलंस का']


Input English sentence: tell her that i am coming
Actual Marathi Translation:  तिला सांग की मी येतेय 
Beam Predicted Marathi Translation: तिला मला विकशील का
Predicted Marathi Translation:  त्याला मला विकशील का 
0.19470019576785122
0


In [1176]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तो तुझा बॉयफ्रेंड आहे का', 'ते तुझा बॉयफ्रेंड आहे का', 'तो तुमचा बॉयफ्रेंड आहे का', 'ते तुमचा बॉयफ्रेंड आहे का', 'ती तुझा बॉयफ्रेंड आहे का', 'तो तुमची बॉयफ्रेंड आहे का', 'तो तुझी बॉयफ्रेंड आहे का', 'ती तुमचा बॉयफ्रेंड आहे का', 'ते तुमची बॉयफ्रेंड आहे का', 'त्या तुझा बॉयफ्रेंड आहे का']


Input English sentence: is that your boyfriend
Actual Marathi Translation:  तो तुझा बॉयफ्रेंड आहे का 
Beam Predicted Marathi Translation: तो तुझा बॉयफ्रेंड आहे का
Predicted Marathi Translation:  तो तुझा बॉयफ्रेंड आहे का 
1.0
1.0


In [1177]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तू उद्या येऊ शकतोस का', 'तू उद्या येऊ शकतेस का', 'तुम्ही उद्या येऊ शकतोस का', 'तू उद्या येऊ शकता का', 'तुम्ही उद्या येऊ शकतेस का', 'तू उद्या यायला शकतोस का', 'तुम्ही उद्या येऊ शकता का', 'तू उद्या येशील शकतोस का', 'तू उद्या यायला शकतेस का', 'तुम्ही उद्या यायला शकतोस का']


Input English sentence: can you come tomorrow
Actual Marathi Translation:  तुम्ही उद्या येऊ शकता का 
Beam Predicted Marathi Translation: तुम्ही उद्या येऊ शकता का
Predicted Marathi Translation:  तू उद्या येऊ शकतोस का 
1.0
0.6


In [1178]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['चूक मला सत्य आला आहे', 'चूक मला चावला आला आहे', 'चूक मला राग आला आहे', 'माझ्या मला सत्य आला आहे', 'चूक मला उशीर आला आहे', 'माझ्या मला चावला आला आहे', 'चूक मला यादीतून आला आहे', 'चूक खरी सत्य आला आहे', 'चूक मला सत्य होती आहे', 'चूक मला सत्य गेली आहे']


Input English sentence: it was actually my fault
Actual Marathi Translation:  खरी तर माझी चूक होती 
Beam Predicted Marathi Translation: चूक खरी सत्य आला आहे
Predicted Marathi Translation:  चूक मला सत्य आला आहे 
0.4
0.2


In [1179]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तिने जेवण फोन केला', 'तिने जेवण फोन पाठलाग', 'तिने मी फोन केला', 'तिने जेवण फोन केलास', 'तिने जेवण मी केला', 'तिने माझं फोन केला', 'तिने जेवण पुस्तक केला', 'तिने मी फोन पाठलाग', 'तिने ते फोन केला', 'तिने मी फोन केलास']


Input English sentence: she cooked the dinner herself
Actual Marathi Translation:  तिने ते जेवण स्वतःच शिजवलं 
Beam Predicted Marathi Translation: तिने जेवण फोन केला
Predicted Marathi Translation:  तिने जेवण फोन केला 
0.38940039153570244
0.38940039153570244


In [1180]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['ते सर्व गरम आहे', 'ते तुझ्यावर गरम आहे', 'ते ठीक गरम आहे', 'तेवढंच सर्व गरम आहे', 'ते तुमच्यावर गरम आहे', 'तेवढंच तुझ्यावर गरम आहे', 'त्या सर्व गरम आहे', 'त्या तुझ्यावर गरम आहे', 'ते सगळी गरम आहे', 'तेवढंच ठीक गरम आहे']


Input English sentence: its all up to you
Actual Marathi Translation:  सगळं तुझ्यावर आहे 
Beam Predicted Marathi Translation: ते तुझ्यावर गरम आहे
Predicted Marathi Translation:  ते सर्व गरम आहे 
0.5
0.25


In [1181]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तू हा स्वेटर घालून बघू', 'तुम्ही हा स्वेटर घालून बघू', 'तू हा ड्रेस घालून बघू', 'तुम्ही हा ड्रेस घालून बघू', 'तू हा स्वेटर वापर बघू', 'तू हा फोटो घालून बघू', 'हा हा स्वेटर घालून बघू', 'तू हे स्वेटर घालून बघू', 'तू हा स्वेटर कमी बघू', 'तुम्ही हा स्वेटर वापर बघू']


Input English sentence: you look better in this dress
Actual Marathi Translation:  तू या ड्रेसमध्ये जास्त चांगली दिसतेस 
Beam Predicted Marathi Translation: तू हा स्वेटर घालून बघू
Predicted Marathi Translation:  तू हा स्वेटर घालून बघू 
0.1637461506155964
0.1637461506155964


In [1182]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तो चंद्रावर उतरणारा पहिला माणूस होता', 'तो ब्रिटिश उतरणारा पहिला माणूस होता', 'तो जपानमध्ये उतरणारा पहिला माणूस होता', 'तो भारतापासून उतरणारा पहिला माणूस होता', 'तो त्याचा उतरणारा पहिला माणूस होता', 'तो जपानला उतरणारा पहिला माणूस होता', 'तो चंद्रावर उतरणारा पहिला म्हणतात होता', 'तो अमेरिकन उतरणारा पहिला माणूस होता', 'तो सगळ्यात उतरणारा पहिला माणूस होता', 'तो चंद्रावर उतरणारा सर्वात माणूस होता']


Input English sentence: he was the first man to land on the moon
Actual Marathi Translation:  तो चंद्रावर उतरणारा पहिला माणूस होता 
Beam Predicted Marathi Translation: तो चंद्रावर उतरणारा पहिला माणूस होता
Predicted Marathi Translation:  तो चंद्रावर उतरणारा पहिला माणूस होता 
1.0
1.0


In [1183]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तिने कादंबर्\u200dया लिहितो', 'तिने कादंबर्\u200dया कादंबर्\u200dया', 'तिने वर्षांमध्ये लिहितो', 'तिने कादंबर्\u200dया लिहिल्या', 'तिने कादंबर्\u200dया लिहितात', 'तिने त्या लिहितो', 'तिने ती लिहितो', 'तिने वर्षांमध्ये कादंबर्\u200dया', 'तिने त्या कादंबर्\u200dया', 'तिने वर्षांमध्ये लिहिल्या']


Input English sentence: she wrote novels in years
Actual Marathi Translation:  तिने वर्षांमध्ये कादंबर्‍या लिहिल्या 
Beam Predicted Marathi Translation: तिने कादंबर्‍या लिहिल्या
Predicted Marathi Translation:  तिने कादंबर्‍या लिहितो 
0.7165313105737893
0.47768754038252614


In [1184]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तू कधी कॅनडाला गेला आहेस का', 'तुम्ही कधी कॅनडाला गेला आहेस का', 'तू कधी कॅनडाला गेले आहेस का', 'तुम्ही कधी कॅनडाला गेले आहेस का', 'तू कधी कॅनडाला झाला आहेस का', 'तुम्ही कधी कॅनडाला झाला आहेस का', 'तू कधी कॅनडाला गेली आहेस का', 'तुम्ही कधी कॅनडाला गेली आहेस का', 'तू कधी कॅनडाला झालेला आहेस का', 'कधी कधी कॅनडाला गेला आहेस का']


Input English sentence: have you ever been to canada
Actual Marathi Translation:  तू कधी कॅनडाला गेला आहेस का 
Beam Predicted Marathi Translation: तू कधी कॅनडाला गेला आहेस का
Predicted Marathi Translation:  तू कधी कॅनडाला गेला आहेस का 
1.0
1.0


In [1185]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मी काल रेडिओ दुरुस्त करत असताना तर वाचलं पण मी एक', 'मी काल रेडिओ एकटाच करत असताना तर वाचलं पण मी एक', 'मी काल रेडिओ करून करत असताना तर वाचलं पण मी एक', 'मी काल रेडिओ दुरुस्त करत असताना तर वाचलं पण ते एक', 'मी काल रेडिओ विकत करत असताना तर वाचलं पण मी एक', 'मी काल रेडिओ दुरुस्त करत असताना तर वाचलं पण तो एक', 'मी काल रेडिओ दुरुस्त करत असताना तर वाचलं पण मी त्यांना', 'मी काल रेडिओ एकटाच करत असताना तर वाचलं पण ते एक', 'मी काल रेडिओ एकटाच करत असताना तर वाचलं पण तो एक', 'मी काल रेडिओ दुरुस्त करत असताना तर वाचलं पण मी तो']


Input English sentence: i spent two hours yesterday trying to fix that broken radio
Actual Marathi Translation:  तो बिघडलेला रेडिओ ठीक करण्यात मी काल दोन तास घालवले 
Beam Predicted Marathi Translation: मी काल रेडिओ दुरुस्त करत असताना तर वाचलं पण तो एक
Predicted Marathi Translation:  मी काल रेडिओ दुरुस्त करत असताना तर वाचलं पण मी एक पु
0.36363636363636365
0.25


In [1186]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['आम्ही काय भेटलो आहोत', 'आम्ही कुठे भेटलो आहोत', 'आपण काय भेटलो आहोत', 'आम्ही काय भेटलो _END', 'आपण कुठे भेटलो आहोत', 'आम्ही कुठे भेटलो _END', 'आपण काय भेटलो _END', 'आम्ही काय कुठे आहोत', 'आपण कुठे भेटलो _END', 'आम्ही काय वाचवलं आहोत']


Input English sentence: where will we meet
Actual Marathi Translation:  आपण कुठे भेटू 
Beam Predicted Marathi Translation: आपण कुठे भेटलो आहोत
Predicted Marathi Translation:  आम्ही काय भेटलो आहोत 
0.5
0


In [1187]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['ती जे गोष्ट काय शिकवू शकत नाही', 'ती जे गोष्ट जे शिकवू शकत नाही', 'ते जे गोष्ट काय शिकवू शकत नाही', 'ती जे गोष्ट तेच शिकवू शकत नाही', 'त्या जे गोष्ट काय शिकवू शकत नाही', 'त्यांना जे गोष्ट काय शिकवू शकत नाही', 'ती जे गोष्ट काय शिकवू शकतो नाही', 'ते जे गोष्ट जे शिकवू शकत नाही', 'ते जे गोष्ट तेच शिकवू शकत नाही', 'त्या जे गोष्ट जे शिकवू शकत नाही']


Input English sentence: i cant understand what she says
Actual Marathi Translation:  ती काय म्हणते मला समजत नाही 
Beam Predicted Marathi Translation: ती जे गोष्ट काय शिकवू शकत नाही
Predicted Marathi Translation:  ती जे गोष्ट काय शिकवू शकत नाही 
0.42857142857142855
0.42857142857142855


In [1188]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मला काहीही खायचं नाहीये', 'मला काहीही खायला नाहीये', 'मला काहीही नको नाहीये', 'मला काहीही विसरायचं नाहीये', 'मला काहीही द्यायचं नाहीये', 'मला काहीही खायचं नाही', 'मला काहीही खायला नाही', 'मला काहीही ऐकायचं नाहीये', 'मला काहीही आठवत नाहीये', 'मला काहीही खाल्लं नाहीये']


Input English sentence: i dont want to forget anything
Actual Marathi Translation:  मला काहीही विसरायचं नाहीये 
Beam Predicted Marathi Translation: मला काहीही विसरायचं नाहीये
Predicted Marathi Translation:  मला काहीही खायचं नाहीये 
1.0
0.75


In [1189]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['बॉस्टनमध्ये इथे', 'बॉस्टनमध्ये पाऊस', 'बॉस्टनमध्ये या', 'बॉस्टनमध्ये सुमारे', 'बॉस्टनमध्ये इथे', 'बॉस्टनमध्ये मुख्य', 'बॉस्टनमध्ये पार्टी', 'बॉस्टनमध्ये आपली', 'बॉस्टनमध्ये कुठेय', 'बॉस्टनमध्ये इथे']


Input English sentence: welcome to boston
Actual Marathi Translation:  बॉस्टनमध्ये तुमचे स्वागत आहे 
Beam Predicted Marathi Translation: बॉस्टनमध्ये इथे
Predicted Marathi Translation:  बॉस्टनमध्ये इथे 
0.18393972058572117
0.18393972058572117


In [1190]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मी तुझ्याशी खोटं बोललो', 'मी तुझ्याशी खोटं बोलले', 'मी तुमच्याशी खोटं बोललो', 'मी तुमच्याशी खोटं बोलले', 'मी तुझ्याशी खोटं बोलला', 'मी तुमच्याशी खोटं बोलला', 'मी तुझ्याशी बोललो बोललो', 'मी तुझ्याशी बोललो बोलले', 'मी तुमच्याशी बोललो बोललो', 'मी तुमच्याशी बोललो बोलले']


Input English sentence: i lied to you
Actual Marathi Translation:  मी तुमच्याशी खोटं बोलले 
Beam Predicted Marathi Translation: मी तुमच्याशी खोटं बोलले
Predicted Marathi Translation:  मी तुझ्याशी खोटं बोललो 
1.0
0.5


In [1191]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मी सँडविच खाते', 'मी सँडविच खात', 'मी एक खाते', 'मी एक खात', 'मी सँडविच खातोय', 'मी सँडविच खाल्लं', 'मी एक खातोय', 'मी सँडविच खायला', 'मी एक खाल्लं', 'मी सँडविच खाल्ला']


Input English sentence: i was eating a sandwich
Actual Marathi Translation:  मी सँडविच खात होते 
Beam Predicted Marathi Translation: मी सँडविच खात
Predicted Marathi Translation:  मी सँडविच खाते 
0.7165313105737893
0.47768754038252614


In [1192]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['हे या टॉमचं पाकीट का', 'हे माझं टॉमचं पाकीट का', 'ही या टॉमचं पाकीट का', 'ही माझं टॉमचं पाकीट का', 'हे या टॉमचं पाकीट विचारा', 'हे या टॉमचं उत्तर का', 'हे माझं टॉमचं पाकीट विचारा', 'हे माझं टॉमचं उत्तर का', 'हे टॉमचं टॉमचं पाकीट का', 'ही या टॉमचं पाकीट विचारा']


Input English sentence: is this real silver
Actual Marathi Translation:  ही खरी चांदी आहे का 
Beam Predicted Marathi Translation: ही या टॉमचं पाकीट का
Predicted Marathi Translation:  हे या टॉमचं पाकीट का 
0.4
0.2


In [1193]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तू दोघं काय करत आहेस', 'तुम्ही दोघं काय करत आहेस', 'तू दोघी काय करत आहेस', 'तुम्ही दोघी काय करत आहेस', 'तू दोघं काय करत आहात', 'तू किती काय करत आहेस', 'तू दोघं किती करत आहेस', 'तुम्ही दोघं काय करत आहात', 'तुम्ही किती काय करत आहेस', 'तू दोघी काय करत आहात']


Input English sentence: what are you two doing
Actual Marathi Translation:  तुम्ही दोघं काय करत आहात 
Beam Predicted Marathi Translation: तुम्ही दोघं काय करत आहात
Predicted Marathi Translation:  तू दोघं काय करत आहेस 
1.0
0.6


In [1194]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
input_sentence = X_train[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_train[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['कश्याचीही काळजी करू नकोस', 'कसलीही काळजी करू नकोस', 'कश्याचीही काळजी करू नका', 'कश्याचीही काळजी करू शकत', 'काळजी काळजी करू नकोस', 'कसलीही काळजी करू नका', 'तुम्हाला काळजी करू नकोस', 'कसलीही काळजी करू शकत', 'तुला काळजी करू नकोस', 'काळजी काळजी करू नका']


Input English sentence: dont you worry about a thing
Actual Marathi Translation:  कसलीही काळजी करू नकोस 
Beam Predicted Marathi Translation: कसलीही काळजी करू नकोस
Predicted Marathi Translation:  कश्याचीही काळजी करू नकोस 
1.0
0.75


## Test Dataset Evaluation

In [1195]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

### Test Experiments
Run the greedy and beam search decoder for 100 examples in the test dataset with varying beam width (k) and show the difference in total BLEU score between the beam search model and greedy model as well as the total time of execution.

In [1319]:
import time
train_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1
n = 2
total_diff = 0
tic = time.perf_counter()
for x in range (0,100):
    k+=1
    (input_seq, actual_output), _ = next(train_gen)
    input_sentence = X_test[k:k+1].values[0]
    decoded_sentence = decode_sequence(input_seq)[:-4]
    beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
    actual_sentence = y_test[k:k+1].values[0][6:-4]
    print('Input English sentence:', input_sentence)
    #print('Actual Marathi Translation:', actual_sentence)
    #print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
    #print('Predicted Marathi Translation:', decoded_sentence)

    beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
    score = calculate_bleu(actual_sentence, decoded_sentence)
    diff = beam_score - score
    total_diff += diff
    #print(beam_score)
    #print(score)
toc = time.perf_counter()
print(n, total_diff)
print(f"Val Time for execution: {toc - tic:0.4f} seconds")

Input English sentence: dont forget your things
Input English sentence: tell her that i am coming
Input English sentence: youre so lazy
Input English sentence: theres got to be something you can do
Input English sentence: there is a right time for everything
Input English sentence: tom is at school now
Input English sentence: i havent seen this movie before
Input English sentence: should i buy a black car or a blue car
Input English sentence: i cry almost every day
Input English sentence: we fixed that
Input English sentence: youre a true patriot
Input English sentence: tom decided to go to boston
Input English sentence: well sing
Input English sentence: tom works as a waiter
Input English sentence: do you smoke cigars
Input English sentence: is french an easy language
Input English sentence: ill get it
Input English sentence: we do not know her
Input English sentence: there are a thousand meters in a kilometer
Input English sentence: he took her aside and told her the news
Input Engli

In [1314]:
import time
train_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1
n = 0
total_diff = 0
tic = time.perf_counter()
for x in range (0,100):
    k+=1
    (input_seq, actual_output), _ = next(train_gen)
    input_sentence = X_test[k:k+1].values[0]
    decoded_sentence = decode_sequence(input_seq)[:-4]
    #beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
    actual_sentence = y_test[k:k+1].values[0][6:-4]
    print('Input English sentence:', input_sentence)
    #print('Actual Mar
    athi Translation:', actual_sentence)
    #print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
    #print('Predicted Marathi Translation:', decoded_sentence)

    #beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
    #score = calculate_bleu(actual_sentence, decoded_sentence)
    #diff = beam_score - score
    #total_diff += diff
    #print(beam_score)
    #print(score)
toc = time.perf_counter()
print(n, total_diff)
print(f"Val Time for execution: {toc - tic:0.4f} seconds")

Input English sentence: dont forget your things
Input English sentence: tell her that i am coming
Input English sentence: youre so lazy
Input English sentence: theres got to be something you can do
Input English sentence: there is a right time for everything
Input English sentence: tom is at school now
Input English sentence: i havent seen this movie before
Input English sentence: should i buy a black car or a blue car
Input English sentence: i cry almost every day
Input English sentence: we fixed that
Input English sentence: youre a true patriot
Input English sentence: tom decided to go to boston
Input English sentence: well sing
Input English sentence: tom works as a waiter
Input English sentence: do you smoke cigars
Input English sentence: is french an easy language
Input English sentence: ill get it
Input English sentence: we do not know her
Input English sentence: there are a thousand meters in a kilometer
Input English sentence: he took her aside and told her the news
Input Engli

### Test Examples - Individual Assessment of sentences

In [1196]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['अजून फोन करून वाजता विसरू नकोस', 'अजून काही करून वाजता विसरू नकोस', 'अजून फोन करून वाजता विसरू नका', 'फोन फोन करून वाजता विसरू नकोस', 'आपली फोन करून वाजता विसरू नकोस', 'अजून फोन करून वाजता का नकोस', 'अजून काही करून वाजता विसरू नका', 'फोन काही करून वाजता विसरू नकोस', 'आपली काही करून वाजता विसरू नकोस', 'अजून काही करून वाजता का नकोस']


Input English sentence: dont forget your things
Actual Marathi Translation:  आपलं सामान विसरू नका 
Beam Predicted Marathi Translation: अजून फोन करून वाजता विसरू नका
Predicted Marathi Translation:  अजून फोन करून वाजता विसरू नकोस 
0.3333333333333333
0.16666666666666669


In [1197]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['त्याला मला विकशील का', 'त्याला मला केलंस का', 'त्याला मला करता का', 'तिला मला विकशील का', 'त्याला मला विकाल का', 'तिला मला केलंस का', 'त्याला मला सांगाल का', 'त्यांना मला विकशील का', 'त्याला मला म्हणतात का', 'त्यांना मला केलंस का']


Input English sentence: tell her that i am coming
Actual Marathi Translation:  त्यांना सांगा की मी येतोय 
Beam Predicted Marathi Translation: त्यांना मला विकशील का
Predicted Marathi Translation:  त्याला मला विकशील का 
0.19470019576785122
0


In [1198]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तू किती आळशी आहेस', 'तू किती मूर्ख आहेस', 'तुम्ही किती आळशी आहेस', 'किती किती आळशी आहेस', 'तू किती आळशी आहात', 'तुम्ही किती मूर्ख आहेस', 'तू किती उद्धट आहेस', 'किती किती मूर्ख आहेस', 'तू किती अभिनेता आहेस', 'तू किती मूर्ख आहात']


Input English sentence: youre so lazy
Actual Marathi Translation:  तू किती आळशी आहेस 
Beam Predicted Marathi Translation: तू किती आळशी आहेस
Predicted Marathi Translation:  तू किती आळशी आहेस 
1.0
1.0


In [1199]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तुला काही करता येईल तर टॉम आहे', 'तुला खायला करता येईल तर टॉम आहे', 'तुला काही करायचा येईल तर टॉम आहे', 'तुला करायला करता येईल तर टॉम आहे', 'तुला खायला करायचा येईल तर टॉम आहे', 'तुम्हाला काही करता येईल तर टॉम आहे', 'तुला करायला करायचा येईल तर टॉम आहे', 'तुला करता करता येईल तर टॉम आहे', 'तुला काही करता येईल याची टॉम आहे', 'तुम्हाला खायला करता येईल तर टॉम आहे']


Input English sentence: theres got to be something you can do
Actual Marathi Translation:  तुला करता येईल असं काहीतरी तर असेलच 
Beam Predicted Marathi Translation: तुला काही करता येईल तर टॉम आहे
Predicted Marathi Translation:  तुला काही करता येईल तर टॉम आहे 
0.5714285714285714
0.5714285714285714


In [1200]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['फक्त एक दिवस आहे', 'किती एक दिवस आहे', 'तिथे एक दिवस आहे', 'किमान एक दिवस आहे', 'फक्त एकदा दिवस आहे', 'अगदी एक दिवस आहे', 'फक्त एक तास आहे', 'काही एक दिवस आहे', 'इथे एक दिवस आहे', 'जरासा एक दिवस आहे']


Input English sentence: there is a right time for everything
Actual Marathi Translation:  प्रत्येक गोष्टीसाठी एक योग्य वेळ असते 
Beam Predicted Marathi Translation: फक्त एक दिवस आहे
Predicted Marathi Translation:  फक्त एक दिवस आहे 
0.15163266492815836
0.15163266492815836


In [1201]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['टॉम आता शाळेत आहे', 'टॉम आता ऑनलाइन आहे', 'टॉम आता तेरा आहे', 'टॉम आता विद्यार्थी आहे', 'टॉम आता तीन आहे', 'टॉम आता घरीच आहे', 'टॉम आता चर्चला आहे', 'टॉम आता शाळा आहे', 'टॉम आता मेरी आहे', 'टॉम आता शाळेला आहे']


Input English sentence: tom is at school now
Actual Marathi Translation:  टॉम आता शाळेत आहे 
Beam Predicted Marathi Translation: टॉम आता शाळेत आहे
Predicted Marathi Translation:  टॉम आता शाळेत आहे 
1.0
1.0


In [1202]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मी हा आधी देवमासा पाहिला नाहीये', 'मी हा चित्रपट देवमासा पाहिला नाहीये', 'मी हा पिक्चर देवमासा पाहिला नाहीये', 'मी हा आधी कधी पाहिला नाहीये', 'मी हा चित्रपट कधी पाहिला नाहीये', 'मी हा पिक्चर कधी पाहिला नाहीये', 'मी हा एका देवमासा पाहिला नाहीये', 'मी हा आधी पिक्चर पाहिला नाहीये', 'मी हा चित्रपट पिक्चर पाहिला नाहीये', 'मी हा पिक्चर पिक्चर पाहिला नाहीये']


Input English sentence: i havent seen this movie before
Actual Marathi Translation:  हा पिक्चर मी याआधी बघितला नाहीये 
Beam Predicted Marathi Translation: मी हा पिक्चर देवमासा पाहिला नाहीये
Predicted Marathi Translation:  मी हा आधी देवमासा पाहिला नाहीये 
0.6666666666666666
0.5


In [1203]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मी एक गाडी विकत घेण्यासाठी गाडी आहे ती एक आहे', 'मी एक गाडी विकत घेण्यासाठी ती आहे ती एक आहे', 'मी एक गाडी विकत घेण्यासाठी गाडी आहे ती एक आहे', 'मी एक गाडी विकत घेण्यासाठी ज्याला आहे ती एक आहे', 'मी एक गाडी गाडी घेण्यासाठी गाडी आहे ती एक आहे', 'मी एक गाडी विकत घेण्यासाठी ती आहे ती एक आहे', 'मी एक गाडी गाडी घेण्यासाठी ती आहे ती एक आहे', 'मी एक गाडी विकत घेण्यासाठी ज्याला आहे ती एक आहे', 'मी एक गाडी गाडी घेण्यासाठी गाडी आहे ती एक आहे', 'मी एक गाडी गाडी घेण्यासाठी ज्याला आहे ती एक आहे']


Input English sentence: should i buy a black car or a blue car
Actual Marathi Translation:  मी काळी गाडी विकत घेऊ का निळी गाडी 
Beam Predicted Marathi Translation: मी एक गाडी विकत घेण्यासाठी गाडी आहे ती एक आहे
Predicted Marathi Translation:  मी एक गाडी विकत घेण्यासाठी गाडी आहे ती एक आहे 
0.4
0.4


In [1204]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मी जवळजवळ दररोज रडतो', 'मी जवळजवळ दररोज खातो', 'मी जवळजवळ दररोज रडते', 'मी जवळजवळ दररोज निघाला', 'मी जवळजवळ दररोजच रडतो', 'मी जवळजवळ दररोज लेखांचा', 'मी जवळजवळ दररोज भेटतो', 'मी जवळजवळ दररोज जेवतो', 'मी जवळजवळ दररोज रडले', 'मी जवळजवळ दररोज शिकार']


Input English sentence: i cry almost every day
Actual Marathi Translation:  मी जवळजवळ दररोज रडतो 
Beam Predicted Marathi Translation: मी जवळजवळ दररोज रडतो
Predicted Marathi Translation:  मी जवळजवळ दररोज रडतो 
1.0
1.0


In [1205]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['आम्ही ते दुरुस्त केलं', 'आपण ते दुरुस्त केलं', 'आम्ही तसं दुरुस्त केलं', 'ते ते दुरुस्त केलं', 'आपण तसं दुरुस्त केलं', 'आम्ही त्यांना दुरुस्त केलं', 'आपण त्यांना दुरुस्त केलं', 'आम्ही त्याला दुरुस्त केलं', 'आम्ही हे दुरुस्त केलं', 'ते तसं दुरुस्त केलं']


Input English sentence: we fixed that
Actual Marathi Translation:  आपण ते दुरुस्त केलं 
Beam Predicted Marathi Translation: आपण ते दुरुस्त केलं
Predicted Marathi Translation:  आम्ही ते दुरुस्त केलं 
1.0
0.75


In [1206]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तू खरे देशभक्त आहेस', 'तुम्ही खरे देशभक्त आहेस', 'तू खरा देशभक्त आहेस', 'तुम्ही खरा देशभक्त आहेस', 'तू ते देशभक्त आहेस', 'तू उद्या देशभक्त आहेस', 'तू खरे नाहीयेस आहेस', 'तुम्ही ते देशभक्त आहेस', 'तुम्ही उद्या देशभक्त आहेस', 'तुम्ही खरे नाहीयेस आहेस']


Input English sentence: youre a true patriot
Actual Marathi Translation:  तू खरी देशभक्त आहेस 
Beam Predicted Marathi Translation: तू खरे देशभक्त आहेस
Predicted Marathi Translation:  तू खरे देशभक्त आहेस 
0.75
0.75


In [1207]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['टॉमने बॉस्टनला जायचं ठरवलं', 'टॉम बॉस्टनला जायचं ठरवलं', 'टॉमने बॉस्टनला बॉस्टनला ठरवलं', 'टॉमने बॉस्टनला जाणार ठरवलं', 'टॉमने बॉस्टनला जायचं नव्हतं', 'टॉमला बॉस्टनला जायचं ठरवलं', 'टॉमने बॉस्टनला यायचं ठरवलं', 'टॉमने बॉस्टनला गेली ठरवलं', 'टॉमने बॉस्टनला जायला ठरवलं', 'टॉमने बॉस्टनला गेलो ठरवलं']


Input English sentence: tom decided to go to boston
Actual Marathi Translation:  टॉमने बॉस्टनला जायचं ठरवलं 
Beam Predicted Marathi Translation: टॉमने बॉस्टनला जायचं ठरवलं
Predicted Marathi Translation:  टॉमने बॉस्टनला जायचं ठरवलं 
1.0
1.0


In [1208]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['आम्ही गाऊ', 'आम्ही गायला', 'आपण गाऊ', 'आपण गायला', 'आम्ही गात', 'आम्ही बरे', 'आपण गात', 'आम्ही _END', 'आपण बरे', 'आपण _END']


Input English sentence: well sing
Actual Marathi Translation:  आम्ही गाऊ 
Beam Predicted Marathi Translation: आम्ही गाऊ
Predicted Marathi Translation:  आम्ही गाऊ 
1.0
1.0


In [1209]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['टॉम एका जवळजवळ बुडून गेला', 'टॉम एका जवळजवळ एका गेला', 'टॉम एका जवळजवळ भरपूर गेला', 'टॉम एका जवळजवळ वस्तुसंग्रहालयात गेला', 'टॉम एका जवळजवळ मासे गेला', 'टॉम एका क्षेत्रात बुडून गेला', 'टॉम एका कंपनीत बुडून गेला', 'टॉम एका क्षेत्रात एका गेला', 'टॉम एका जवळजवळ बहुधा गेला', 'टॉम एका क्षेत्रात भरपूर गेला']


Input English sentence: tom works as a waiter
Actual Marathi Translation:  टॉम वेटरची नोकरी करतो 
Beam Predicted Marathi Translation: टॉम एका जवळजवळ बुडून गेला
Predicted Marathi Translation:  टॉम एका जवळजवळ बुडून गेला 
0.2
0.2


In [1210]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['सिगार ओढतेस का', 'सिगार ओढतोस का', 'सिगार ओढता का', 'सिगार खेळतोस का', 'सिगार का का', 'सिगार खेळतेस का', 'सिगार उघडाल का', 'सिगार उघडू का', 'सिगार उघडशील का', 'सिगार जबाबदार का']


Input English sentence: do you smoke cigars
Actual Marathi Translation:  सिगार ओढतेस का 
Beam Predicted Marathi Translation: सिगार ओढतेस का
Predicted Marathi Translation:  सिगार ओढतेस का 
1.0
1.0


In [1211]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['फ्रेंच ही सोपी भाषा आहे', 'फ्रेंच ही भाषा भाषा आहे', 'फ्रेंच ही फ्रेंच भाषा आहे', 'फ्रेंच ही मोठी भाषा आहे', 'फ्रेंच ही जास्त भाषा आहे', 'फ्रेंच ही सोपी भाषा आहे', 'फ्रेंच ही भाषा भाषा आहे', 'फ्रेंच ही चूक भाषा आहे', 'फ्रेंच सोपी सोपी भाषा आहे', 'फ्रेंच सोपी भाषा भाषा आहे']


Input English sentence: is french an easy language
Actual Marathi Translation:  फ्रेंच ही सोपी भाषा आहे का 
Beam Predicted Marathi Translation: फ्रेंच ही सोपी भाषा आहे
Predicted Marathi Translation:  फ्रेंच ही सोपी भाषा आहे 
0.8187307530779819
0.8187307530779819


In [1212]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मी हवा', 'मी आवडतो', 'मी आवडतं', 'मी घेईन', 'मी आणतो', 'मी तुम्हाला', 'मी तुला', 'मी आवडते', 'मी आवडलं', 'मी समजतं']


Input English sentence: ill get it
Actual Marathi Translation:  मी उघडते 
Beam Predicted Marathi Translation: मी हवा
Predicted Marathi Translation:  मी हवा 
0.5
0.5


In [1213]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['आम्ही त्यांना ओळखत नाही', 'आपण त्यांना ओळखत नाही', 'आम्ही त्याला ओळखत नाही', 'आपण त्याला ओळखत नाही', 'आम्ही हे ओळखत नाही', 'आपण हे ओळखत नाही', 'आम्ही त्यांना माहीत नाही', 'आम्ही तिला ओळखत नाही', 'आपण त्यांना माहीत नाही', 'आम्ही तसं ओळखत नाही']


Input English sentence: we do not know her
Actual Marathi Translation:  आम्ही तिला ओळखत नाही 
Beam Predicted Marathi Translation: आम्ही तिला ओळखत नाही
Predicted Marathi Translation:  आम्ही त्यांना ओळखत नाही 
1.0
0.75


In [1214]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['एका किलोमीटरमध्ये एक हजार मीटर असतात', 'एका हजार एक हजार मीटर असतात', 'एका हातात एक हजार मीटर असतात', 'एका अमेरिकन एक हजार मीटर असतात', 'एका चौरस एक हजार मीटर असतात', 'एका बेटावर एक हजार मीटर असतात', 'एका शब्दात एक हजार मीटर असतात', 'एका किलोमीटरमध्ये एक हजार येन असतात', 'एका मोठ्या एक हजार मीटर असतात', 'एका कवितेचा एक हजार मीटर असतात']


Input English sentence: there are a thousand meters in a kilometer
Actual Marathi Translation:  एका किलोमीटरमध्ये एक हजार मीटर असतात 
Beam Predicted Marathi Translation: एका किलोमीटरमध्ये एक हजार मीटर असतात
Predicted Marathi Translation:  एका किलोमीटरमध्ये एक हजार मीटर असतात 
1.0
1.0


In [1215]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तिने त्याला ठार बाजूला सांगितलं नाहीत ते धावू', 'तिने त्याला ठार बाजूला सांगितलं नाहीत ते धावू', 'तिने त्याला ठार बाजूला सांगितलं म्हणून ते धावू', 'तिने त्याला ठार बाजूला सांगितलं नाहीत ती धावू', 'तिने त्याला ठार माणसाशी सांगितलं नाहीत ते धावू', 'तिने त्याला ठार बाजूला सांगितलं नाहीत ते धावू', 'त्यांनी त्याला ठार बाजूला सांगितलं नाहीत ते धावू', 'तिने त्याला ठार बाजूला सांगितलं नाहीत त्या धावू', 'तिने त्याला ठार बाजूला त्याला नाहीत ते धावू', 'तिने त्याला ठार बाजूला सांगितलं नाहीत तो धावू']


Input English sentence: he took her aside and told her the news
Actual Marathi Translation:  त्याने त्यांना बाजूला करून त्यांना बातमी सांगितली 
Beam Predicted Marathi Translation: तिने त्याला ठार बाजूला सांगितलं नाहीत ते धावू
Predicted Marathi Translation:  तिने त्याला ठार बाजूला सांगितलं नाहीत ते धावू न
0.12500000000000003
0.11111111111111109


In [1216]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['आपल्याला टॉमसारख्या अधिक लोकांची गरज आहे', 'आम्हाला टॉमसारख्या अधिक लोकांची गरज आहे', 'आपल्याला अजून अधिक लोकांची गरज आहे', 'आम्हाला अजून अधिक लोकांची गरज आहे', 'आपल्याला टॉमसारख्या अजून लोकांची गरज आहे', 'आम्हाला टॉमसारख्या अजून लोकांची गरज आहे', 'आपल्याला टॉमसारख्या अधिक बर्\u200dयापैकी गरज आहे', 'आम्हाला टॉमसारख्या अधिक बर्\u200dयापैकी गरज आहे', 'आपल्याला अजून अजून लोकांची गरज आहे', 'आम्हाला अजून अजून लोकांची गरज आहे']


Input English sentence: we need more people like tom
Actual Marathi Translation:  आपल्याला टॉमसारख्या अधिक लोकांची गरज आहे 
Beam Predicted Marathi Translation: आपल्याला टॉमसारख्या अधिक लोकांची गरज आहे
Predicted Marathi Translation:  आपल्याला टॉमसारख्या अधिक लोकांची गरज आहे 
1.0
1.0


In [1217]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['आमच्याकडे एक कप पाणी आहे', 'आमच्याकडे जरासं कप पाणी आहे', 'आमच्याकडे एक कप पाणी होतं', 'आपल्याकडे एक कप पाणी आहे', 'आमच्याकडे एक स्कर्ट पाणी आहे', 'आमच्याकडे जरासं कप पाणी होतं', 'आपल्याकडे जरासं कप पाणी आहे', 'आमच्याकडे जरासं स्कर्ट पाणी आहे', 'आपल्याकडे एक कप पाणी होतं', 'आमच्याकडे एक स्कर्ट पाणी होतं']


Input English sentence: we had a little water
Actual Marathi Translation:  आमच्याकडे थोडंसं पाणी होतं 
Beam Predicted Marathi Translation: आमच्याकडे एक कप पाणी होतं
Predicted Marathi Translation:  आमच्याकडे एक कप पाणी आहे 
0.6
0.4


In [1218]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तू घरी आपल्या बायकोबरोबर असायला हवं', 'तू तुझ्या आपल्या बायकोबरोबर असायला हवं', 'तू घरी नाताळाची बायकोबरोबर असायला हवं', 'तू तुझ्या नाताळाची बायकोबरोबर असायला हवं', 'तू घरी तुझ्या बायकोबरोबर असायला हवं', 'तू तुझ्या तुझ्या बायकोबरोबर असायला हवं', 'तू घरी घर बायकोबरोबर असायला हवं', 'तू तुझ्या घर बायकोबरोबर असायला हवं', 'तुझ्या घरी आपल्या बायकोबरोबर असायला हवं', 'तुम्ही घरी आपल्या बायकोबरोबर असायला हवं']


Input English sentence: you should be at home with your wife
Actual Marathi Translation:  तुला घरी तुझ्या बायकोबरोबर असायला हवं 
Beam Predicted Marathi Translation: तू घरी तुझ्या बायकोबरोबर असायला हवं
Predicted Marathi Translation:  तू घरी आपल्या बायकोबरोबर असायला हवं 
0.8333333333333334
0.6666666666666666


In [1219]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तू आज अभ्यास करत आहेस का', 'तू इथे अभ्यास करत आहेस का', 'तुम्ही आज अभ्यास करत आहेस का', 'तू आज अभ्यास करत आहेस ना', 'तुम्ही इथे अभ्यास करत आहेस का', 'तू आज अभ्यास करत होतीस का', 'तू इथे अभ्यास करत आहेस ना', 'तू आज अभ्यास करत होतास का', 'तू आज तसं करत आहेस का', 'तू आज होतीस करत आहेस का']


Input English sentence: youre happy here arent you
Actual Marathi Translation:  तू इथे खूश आहेस ना 
Beam Predicted Marathi Translation: तू इथे अभ्यास करत आहेस ना
Predicted Marathi Translation:  तू आज अभ्यास करत आहेस का 
0.6666666666666666
0.3333333333333333


In [1220]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['टॉमने खाली आला नाही', 'टॉमने सुरुवात आला नाही', 'टॉमने खाली केलं नाही', 'टॉमने खाली नाही नाही', 'टॉमने सुरुवात केलं नाही', 'टॉमने सुरुवात नाही नाही', 'टॉमने सुरू आला नाही', 'टॉमने खाली राहिला नाही', 'टॉमने खाली नव्हती नाही', 'टॉमने सुरुवात राहिला नाही']


Input English sentence: tom started reading
Actual Marathi Translation:  टॉमने वाचायला सुरुवात केली 
Beam Predicted Marathi Translation: टॉमने सुरुवात आला नाही
Predicted Marathi Translation:  टॉमने खाली आला नाही 
0.5
0.25


In [1221]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['कपडे काढा', 'कपडे काढ', 'कपडे घ्या', 'तुझे काढा', 'तुमचे काढा', 'तुझे काढ', 'तुमचे काढ', 'कपडे घे', 'खिश्यात काढा', 'तुझा काढा']


Input English sentence: take off your clothes
Actual Marathi Translation:  कपडे काढ 
Beam Predicted Marathi Translation: कपडे काढ
Predicted Marathi Translation:  कपडे काढा 
1.0
0.5


In [1222]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['आम्ही त्यांना पाहिलं आहे', 'आम्ही त्यांना बघितलं आहे', 'आपण त्यांना पाहिलं आहे', 'आपण त्यांना बघितलं आहे', 'आम्ही तिला पाहिलं आहे', 'आम्ही तिला बघितलं आहे', 'आम्ही त्याला पाहिलं आहे', 'आम्ही त्याला बघितलं आहे', 'आपण तिला पाहिलं आहे', 'आपण तिला बघितलं आहे']


Input English sentence: weve seen her
Actual Marathi Translation:  आम्ही तिला पाहिलं आहे 
Beam Predicted Marathi Translation: आम्ही तिला पाहिलं आहे
Predicted Marathi Translation:  आम्ही त्यांना पाहिलं आहे 
1.0
0.75


In [1223]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['टॉम कुठे जन्माला आला होता', 'टॉम कुठे जन्माला आले होता', 'टॉम कुठे जन्मलेला आला होता', 'टॉम कुठे जन्माला आलो होता', 'टॉम कुठे जन्मलेले आला होता', 'टॉम कुठे जन्मलेला आले होता', 'टॉम कुठे जन्मलेले आले होता', 'टॉम कुठे जन्मलेला आलो होता', 'टॉम कुठे जन्माला आला _END', 'टॉम कुठे जन्माला गेला होता']


Input English sentence: where was tom born
Actual Marathi Translation:  टॉम कुठे जन्मलेले 
Beam Predicted Marathi Translation: टॉम कुठे जन्मलेले आला होता
Predicted Marathi Translation:  टॉम कुठे जन्माला आला होता 
0.6
0.4


In [1224]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['टॉम माझा बॉयफ्रेंड तेव्हा मी संगीतकार आहे', 'टॉम माझा बॉयफ्रेंड होता मी संगीतकार आहे', 'टॉम माझा बॉयफ्रेंड जिथे मी संगीतकार आहे', 'टॉम माझा बॉयफ्रेंड तेव्हा माझा संगीतकार आहे', 'टॉम माझा बॉयफ्रेंड होता माझा संगीतकार आहे', 'टॉम माझा बॉयफ्रेंड जिथे माझा संगीतकार आहे', 'टॉम माझा बॉयफ्रेंड तेव्हा पूर्णपणे संगीतकार आहे', 'टॉम माझा बॉयफ्रेंड तेव्हा कोण संगीतकार आहे', 'टॉम माझा बॉयफ्रेंड तेव्हा वेडा संगीतकार आहे', 'टॉम माझा बॉयफ्रेंड कुठे मी संगीतकार आहे']


Input English sentence: tom was my first boyfriend
Actual Marathi Translation:  टॉम माझा पहिला बॉयफ्रेंड होता 
Beam Predicted Marathi Translation: टॉम माझा बॉयफ्रेंड होता मी संगीतकार आहे
Predicted Marathi Translation:  टॉम माझा बॉयफ्रेंड तेव्हा मी संगीतकार आहे 
0.5714285714285714
0.42857142857142855


In [1225]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['ड्वाइट आइसेनहोवर साली राज्याध्यक्ष म्हणून निवडून', 'ड्वाइट आइसेनहोवर साली राज्याध्यक्ष म्हणून निवडून', 'ड्वाइट आइसेनहोवर साली साली म्हणून निवडून', 'ड्वाइट आइसेनहोवर साली साली म्हणून निवडून', 'ड्वाइट आइसेनहोवर साली पतंग म्हणून निवडून', 'ड्वाइट आइसेनहोवर साली येथे म्हणून निवडून', 'ड्वाइट आइसेनहोवर साली चर्चमध्ये म्हणून निवडून', 'ड्वाइट आइसेनहोवर साली चोराला म्हणून निवडून', 'ड्वाइट आइसेनहोवर साली रशियाने म्हणून निवडून', 'ड्वाइट आइसेनहोवर साली पतंग म्हणून निवडून']


Input English sentence: dwight eisenhower was elected president in
Actual Marathi Translation:  ड्वाइट आइसेनहोवर साली राज्याध्यक्ष म्हणून निवडून आले 
Beam Predicted Marathi Translation: ड्वाइट आइसेनहोवर साली राज्याध्यक्ष म्हणून निवडून
Predicted Marathi Translation:  ड्वाइट आइसेनहोवर साली राज्याध्यक्ष म्हणून निवडून
0.846481724890614
0.846481724890614


In [1226]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तुमच्या वर्गात किती मुलं आहेत', 'तुझ्या वर्गात किती मुलं आहेत', 'तुमच्या वर्गात मुली मुलं आहेत', 'तुमच्या वर्गात मुलं मुलं आहेत', 'तुझ्या वर्गात मुली मुलं आहेत', 'तुझ्या वर्गात मुलं मुलं आहेत', 'तुमच्या मुली किती मुलं आहेत', 'तुमच्या वर्गात कोणत्या मुलं आहेत', 'तुमच्या वर्गात कोणती मुलं आहेत', 'तुमच्या वर्गात कुठे मुलं आहेत']


Input English sentence: how many boys are in your class
Actual Marathi Translation:  तुमच्या वर्गात मुलं किती आहेत 
Beam Predicted Marathi Translation: तुमच्या वर्गात किती मुलं आहेत
Predicted Marathi Translation:  तुमच्या वर्गात किती मुलं आहेत 
1.0
1.0


In [1227]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['इंग्रजी बोलू देणं अशक्य होतं', 'इंग्रजी बोलू देणं कठीण होतं', 'लहान बोलू देणं अशक्य होतं', 'इंग्रजी बोलू वाचता अशक्य होतं', 'लहान बोलू देणं कठीण होतं', 'इंग्रजी बोलू वाचता कठीण होतं', 'इंग्रजी वाचता देणं अशक्य होतं', 'लहान बोलू वाचता अशक्य होतं', 'इंग्रजी बोलणं देणं अशक्य होतं', 'इंग्रजी बोलू देणं अशक्य _END']


Input English sentence: is it hard to speak english
Actual Marathi Translation:  इंग्रजी बोलणं कठीण असतं का 
Beam Predicted Marathi Translation: इंग्रजी बोलू देणं कठीण होतं
Predicted Marathi Translation:  इंग्रजी बोलू देणं अशक्य होतं 
0.4
0.2


In [1228]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['त्यांना काय केस हवा होता', 'त्याला काय केस हवा होता', 'त्यांना परग्रहवासी केस हवा होता', 'त्यांना काय आवडतो हवा होता', 'त्यांना कोणत्या केस हवा होता', 'त्याला परग्रहवासी केस हवा होता', 'त्याला काय आवडतो हवा होता', 'त्यांना काय कपडे हवा होता', 'त्यांना काय शत्रू हवा होता', 'त्यांना परग्रहवासी आवडतो हवा होता']


Input English sentence: what is he after
Actual Marathi Translation:  त्याला काय हवं आहे 
Beam Predicted Marathi Translation: त्याला काय केस हवा होता
Predicted Marathi Translation:  त्यांना काय केस हवा होता 
0.4
0.2


In [1229]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तो टीव्ही बघतोय', 'ते टीव्ही बघतोय', 'तो टीव्ही बघत', 'ते टीव्ही बघत', 'तो टीव्हीवर बघतोय', 'ते टीव्हीवर बघतोय', 'तो टीव्हीवर बघत', 'ते टीव्हीवर बघत', 'त्या टीव्ही बघतोय', 'त्या टीव्ही बघत']


Input English sentence: he is watching tv
Actual Marathi Translation:  तो टीव्ही बघतोय 
Beam Predicted Marathi Translation: तो टीव्ही बघतोय
Predicted Marathi Translation:  तो टीव्ही बघतोय 
1.0
1.0


In [1230]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मी टॉमला जवळजवळ दररोज फोन करू शकतो', 'मी टॉमला आज दररोज फोन करू शकतो', 'मी टॉमला जवळजवळ दररोज फोन करून शकतो', 'मी टॉमला जवळजवळ रोजच फोन करू शकतो', 'मी टॉमला जवळजवळ दररोज आणि करू शकतो', 'मी टॉमला आज दररोज फोन करून शकतो', 'मी टॉमला आज रोजच फोन करू शकतो', 'मी टॉमला आज दररोज आणि करू शकतो', 'मी टॉमला जवळजवळ रोजच फोन करून शकतो', 'मी टॉमला जवळजवळ दररोज पुन्हा करू शकतो']


Input English sentence: i call tom almost every evening
Actual Marathi Translation:  टॉमला मी जवळजवळ प्रत्येक संध्याकाळी फोन करते 
Beam Predicted Marathi Translation: मी टॉमला जवळजवळ दररोज फोन करू शकतो
Predicted Marathi Translation:  मी टॉमला जवळजवळ दररोज फोन करू शकतो 
0.5714285714285714
0.5714285714285714


In [1231]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मला तुमची खूप आठवण येईल', 'मला तुझी खूप आठवण येईल', 'मला तुझा खूप आठवण येईल', 'मला अगदी खूप आठवण येईल', 'मला तुमची खूप आठवण येते', 'मला तू खूप आठवण येईल', 'मला तुझी खूप आठवण येते', 'मला तुमची खूप आठवण येणार', 'मला तुमची आठवण आठवण येईल', 'मला तुमची वेळ आठवण येईल']


Input English sentence: ill miss you very much
Actual Marathi Translation:  मला तुझी खूप आठवण येईल 
Beam Predicted Marathi Translation: मला तुझी खूप आठवण येईल
Predicted Marathi Translation:  मला तुमची खूप आठवण येईल 
1.0
0.8


In [1232]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['आता हा सगळं बदललं आहे', 'आता या सगळं बदललं आहे', 'आता हे सगळं बदललं आहे', 'आता हा आता बदललं आहे', 'आता या आता बदललं आहे', 'आता हा कोणाचा बदललं आहे', 'आता या कोणाचा बदललं आहे', 'हे हा सगळं बदललं आहे', 'हे या सगळं बदललं आहे', 'आता हे आता बदललं आहे']


Input English sentence: all this has now changed
Actual Marathi Translation:  आता हे सगळं बदललं आहे 
Beam Predicted Marathi Translation: आता हे सगळं बदललं आहे
Predicted Marathi Translation:  आता हा सगळं बदललं आहे 
1.0
0.8


In [1233]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मी टॉमचा सेक्रेटरी होतो', 'मी टॉमचा सेक्रेटरी होते', 'मी टॉमची सेक्रेटरी होतो', 'मी टॉमचा खून होतो', 'मी टॉमची सेक्रेटरी होते', 'मी टॉमचा खून होते', 'मी टॉमच्या सेक्रेटरी होतो', 'मी टॉमची खून होतो', 'मी टॉमचा कुत्रा होतो', 'मी टॉमच्या सेक्रेटरी होते']


Input English sentence: i was toms secretary
Actual Marathi Translation:  मी टॉमची सेक्रेटरी होते 
Beam Predicted Marathi Translation: मी टॉमची सेक्रेटरी होते
Predicted Marathi Translation:  मी टॉमचा सेक्रेटरी होतो 
1.0
0.5


In [1234]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तू कधी किडा दिसून येतं', 'तू कधी स्वतः दिसून येतं', 'तुम्ही कधी किडा दिसून येतं', 'तुम्ही कधी स्वतः दिसून येतं', 'तू कधी किडा रेडिओ येतं', 'तू कधी रेडिओ दिसून येतं', 'तू कधी किडा ऐकलीस येतं', 'तू कधी किडा चित्रपट येतं', 'तू कधी चित्र दिसून येतं', 'तू कधी किडा खाल्ला येतं']


Input English sentence: when did you come to japan
Actual Marathi Translation:  तुम्ही जपानला कधी आलात 
Beam Predicted Marathi Translation: तुम्ही कधी किडा दिसून येतं
Predicted Marathi Translation:  तू कधी किडा दिसून येतं 
0.4
0.2


In [1235]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['टॉम एक आहे', 'टॉम एक आहे', 'टॉम मोठा आहे', 'टॉम एक जुना', 'टॉम लाजाळू आहे', 'टॉम दुष्ट आहे', 'टॉम एक खेळ', 'टॉम हा आहे', 'टॉम जवळच आहे', 'टॉम एकच आहे']


Input English sentence: tom is a sociologist
Actual Marathi Translation:  टॉम समाजशास्त्रज्ञ आहे 
Beam Predicted Marathi Translation: टॉम एक आहे
Predicted Marathi Translation:  टॉम एक आहे 
0.6666666666666666
0.6666666666666666


In [1236]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तू कैदी आहेस का', 'तुम्ही कैदी आहेस का', 'तू कैदी आहात का', 'तुम्ही कैदी आहात का', 'कैदी कैदी आहेस का', 'कैदी कैदी आहात का', 'तू कैदी का का', 'तुझा कैदी आहेस का', 'तुम्ही कैदी का का', 'तुमचे कैदी आहेस का']


Input English sentence: are you a prisoner
Actual Marathi Translation:  तुम्ही कैदी आहात का 
Beam Predicted Marathi Translation: तुम्ही कैदी आहात का
Predicted Marathi Translation:  तू कैदी आहेस का 
1.0
0.5


In [1237]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['लहान मुलांना खूप दूर काम करू', 'लहान मुलांना अतिशय दूर काम करू', 'लहान मुलांना खूप दूर शांत करू', 'लहान मुलांना खूप दूर संपली करू', 'लहान मुलांना खूप दूर _END करू', 'लहान मुलांना खूप दूर काम कर', 'लहान मुलांना आवाज दूर काम करू', 'लहान मुलांना खूप दूर अतिशय करू', 'लहान मुलांना खूप दूर चालत करू', 'लहान मुलांना अतिशय दूर शांत करू']


Input English sentence: the children are making a lot of noise
Actual Marathi Translation:  मुलं खूप आवाज करताहेत 
Beam Predicted Marathi Translation: लहान मुलांना खूप दूर काम करू
Predicted Marathi Translation:  लहान मुलांना खूप दूर काम करू 
0.16666666666666669
0.16666666666666669


In [1238]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['तू माझ्याशी खोटं बोलू शकत नाहीस', 'तू माझ्याशी खोटं बोलू नकोस नाहीस', 'तू माझ्याशी खोटं बोलू शकत नाहीस', 'तू माझ्याशी खोटं बोलू नकोस नाहीस', 'तू माझ्याशी खोटं बोलू नका नाहीस', 'तू माझ्याशी खोटं बोलू नका नाहीस', 'तू माझ्याशी खोटं बोलू शकत नाही', 'तू माझ्याशी खोटं बोलू नकोस नाही', 'तू माझ्याशी खोटं बोलत शकत नाहीस', 'तू माझ्याशी खोटं बोलत नकोस नाहीस']


Input English sentence: dont you lie to me
Actual Marathi Translation:  तू माझ्याशी खोटं बोलू नकोस 
Beam Predicted Marathi Translation: तू माझ्याशी खोटं बोलू नकोस नाहीस
Predicted Marathi Translation:  तू माझ्याशी खोटं बोलू शकत नाहीस 
0.8333333333333334
0.6666666666666666


In [1239]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['टॉम इथे काम करत नाही', 'टॉम इथे काम करत नाहीये', 'टॉम इथे काम करत नव्हती', 'टॉम इथे काम करत होती', 'टॉम इथे काम करत नाहीत', 'टॉम इथे काम करत आहे', 'टॉम इथेच काम करत नाही', 'टॉम इथे काम करत नव्हता', 'टॉम काम काम करत नाही', 'टॉम येथे काम करत नाही']


Input English sentence: tom doesnt work here
Actual Marathi Translation:  टॉम इथे काम करत नाही 
Beam Predicted Marathi Translation: टॉम इथे काम करत नाही
Predicted Marathi Translation:  टॉम इथे काम करत नाही 
1.0
1.0


In [1240]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['एक्स्टर्नल हार्ड रेकॉर्डर माहिती होतं', 'टेप हार्ड रेकॉर्डर माहिती होतं', 'एक्स्टर्नल हार्ड रेकॉर्डर माहिती होता', 'टेप हार्ड रेकॉर्डर माहिती होता', 'एक्स्टर्नल हार्ड ड्राइव्हवर माहिती होतं', 'केलान्तान हार्ड रेकॉर्डर माहिती होतं', 'टेप हार्ड ड्राइव्हवर माहिती होतं', 'एक्स्टर्नल हार्ड रेकॉर्डर माहिती शिकवली', 'टेप हार्ड रेकॉर्डर माहिती शिकवली', 'एक्स्टर्नल हार्ड रेकॉर्डर माहिती असते']


Input English sentence: the government was earning more money than it needed
Actual Marathi Translation:  शासन गरजेपेक्षा जास्त पैसे कमवत होतं 
Beam Predicted Marathi Translation: एक्स्टर्नल हार्ड रेकॉर्डर माहिती होतं
Predicted Marathi Translation:  एक्स्टर्नल हार्ड रेकॉर्डर माहिती होतं 
0.1637461506155964
0.1637461506155964


In [1241]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['रेडिओ बंद कर', 'रेडिओ बंद करा', 'रेडिओचा बंद कर', 'लाईट बंद कर', 'रेडिओचा बंद करा', 'लाईट बंद करा', 'टीव्हीचा बंद कर', 'टीव्ही बंद कर', 'खिडकी बंद कर', 'रेडियो बंद कर']


Input English sentence: turn off the radio
Actual Marathi Translation:  रेडिओ बंद करा 
Beam Predicted Marathi Translation: रेडिओ बंद करा
Predicted Marathi Translation:  रेडिओ बंद कर 
1.0
0.6666666666666666


In [1242]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['टॉमने एक एक डझन एक दिवसांनंतर व गमवून बसला', 'टॉमने एक एक डझन एक दिवसांनंतर व तो बसला', 'टॉमने एक एक डझन एक दिवसांनंतर की गमवून बसला', 'टॉमने एक नवीन डझन एक दिवसांनंतर व गमवून बसला', 'टॉमने एक एक डझन एक दिवसांनंतर की तो बसला', 'टॉमने एक नवीन डझन एक दिवसांनंतर व तो बसला', 'टॉमने एक एक डझन एक दिवसांनंतर व दिले बसला', 'टॉमने एक नवीन डझन एक दिवसांनंतर की गमवून बसला', 'टॉमने एक एक डझन एक दिवसांनंतर व त्याला बसला', 'टॉमने एक नवीन डझन एक दिवसांनंतर की तो बसला']


Input English sentence: tom bought a rose and gave it to mary
Actual Marathi Translation:  टॉमने एक गुलाब विकत घेतला आणि तो मेरीला दिला 
Beam Predicted Marathi Translation: टॉमने एक एक डझन एक दिवसांनंतर व तो बसला
Predicted Marathi Translation:  टॉमने एक एक डझन एक दिवसांनंतर व गमवून बसला 
0.3333333333333333
0.2222222222222222


In [1243]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['फ्रान्स इंग्लंडच्या दक्षिणेकडे आहे', 'रक्त इंग्लंडच्या दक्षिणेकडे आहे', 'वेलिंग्टन इंग्लंडच्या दक्षिणेकडे आहे', 'यूएसएच्या इंग्लंडच्या दक्षिणेकडे आहे', 'जपानची इंग्लंडच्या दक्षिणेकडे आहे', 'फ्रान्स इंग्लंडच्या बुडत आहे', 'नापोली इंग्लंडच्या दक्षिणेकडे आहे', 'फ्रान्स फ्रान्स दक्षिणेकडे आहे', 'फ्रान्स आर्थिक दक्षिणेकडे आहे', 'फ्रान्स राष्ट्रीय दक्षिणेकडे आहे']


Input English sentence: france is to the south of england
Actual Marathi Translation:  फ्रान्स इंग्लंडच्या दक्षिणेकडे आहे 
Beam Predicted Marathi Translation: फ्रान्स इंग्लंडच्या दक्षिणेकडे आहे
Predicted Marathi Translation:  फ्रान्स इंग्लंडच्या दक्षिणेकडे आहे 
1.0
1.0


In [1244]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
input_sentence = X_test[k:k+1].values[0]
decoded_sentence = decode_sequence(input_seq)[:-4]
beam_decoded_sentences = beam_search_decode_sequence(input_seq, n)
actual_sentence = y_test[k:k+1].values[0][6:-4]
print('Input English sentence:', input_sentence)
print('Actual Marathi Translation:', actual_sentence)
print('Beam Predicted Marathi Translation:', best_beam(actual_sentence, beam_decoded_sentences))
print('Predicted Marathi Translation:', decoded_sentence)

beam_score = calculate_bleu_for_beam(actual_sentence, beam_decoded_sentences)
score = calculate_bleu(actual_sentence, decoded_sentence)
print(beam_score)
print(score)

Top Sentences
['मी टॉमला टीव्हीवर अनेकदा पाहिलं', 'मी तो टीव्हीवर अनेकदा पाहिलं', 'मी टॉमला एका अनेकदा पाहिलं', 'मी तो एका अनेकदा पाहिलं', 'मी उन्हाळ्यात टीव्हीवर अनेकदा पाहिलं', 'मी उन्हाळ्यात एका अनेकदा पाहिलं', 'मी बॉस्टनला टीव्हीवर अनेकदा पाहिलं', 'मी बॉस्टनला एका अनेकदा पाहिलं', 'मी बॉस्टनमध्ये टीव्हीवर अनेकदा पाहिलं', 'मी बॉस्टनमध्ये एका अनेकदा पाहिलं']


Input English sentence: ive seen tom on tv many times
Actual Marathi Translation:  मी टॉमला टीव्हीवर भरपूर वेळा पाहिलं आहे 
Beam Predicted Marathi Translation: मी टॉमला टीव्हीवर अनेकदा पाहिलं
Predicted Marathi Translation:  मी टॉमला टीव्हीवर अनेकदा पाहिलं 
0.5362560368285115
0.5362560368285115
